## *Git-Hub Link*

# *Streamlit App Link*

# ***FBI Crime Record Analysis: Model Training, Forecasting & Deployment***

## Project Introduction:

This project focuses on analyzing historical FBI crime data to identify trends, patterns, and seasonal variations across different crime types and neighbourhoods, and to forecast future crime counts using both machine learning and time series forecasting models.

The key objectives of the project are:

Analyze historical crime trends over multiple years.

Identify seasonal and long-term patterns in crime occurrences.

Build forecasting models to predict future crime counts.

Compare performance of advanced machine learning models (LightGBM, XGBoost) and statistical time series models (SARIMAX).

Forecast crime counts specifically for the years 2012 and 2013.

Provide insights and recommendations for crime prevention and resource allocation.

The project uses:

Feature engineering techniques

Lag features and rolling statistics

Machine learning forecasting framework (MLForecast)

High Performance Gradient Boosted Decsision Tree Models (LightGBM and XGBoost)

Statistical time series modeling (SARIMAX)

Visualization of historical and forecasted trends

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')

In [2]:
path_1 = '/content/drive/MyDrive/Colab Notebooks/My Colab Notebooks/Module 6/Copy of Train.xlsx'
path_2 = '/content/drive/MyDrive/Colab Notebooks/My Colab Notebooks/Module 6/Copy of Test.csv'

In [3]:
import pandas as pd

# Load the training dataset from an Excel file.
# The file path is defined in an earlier cell as `path_1`.
train_df = pd.read_excel(path_1)

# Display the first few rows of the loaded DataFrame to inspect its structure and content.
train_df.head()

,TYPE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y,Latitude,Longitude,HOUR,MINUTE,YEAR,MONTH,DAY,Date
0,Other Theft,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.269802,-123.083763,16.0,15.0,1999,5,12,1999-05-12
1,Other Theft,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.269802,-123.083763,15.0,20.0,1999,5,7,1999-05-07
2,Other Theft,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.269802,-123.083763,16.0,40.0,1999,4,23,1999-04-23
3,Other Theft,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.269802,-123.083763,11.0,15.0,1999,4,20,1999-04-20
4,Other Theft,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.269802,-123.083763,17.0,45.0,1999,4,12,1999-04-12


In [4]:
# Display the value counts of the 'NEIGHBOURHOOD' column in the training DataFrame.
# This helps in understanding the distribution of crimes across different neighborhoods and identifying potential categorical issues.
train_df['NEIGHBOURHOOD'].value_counts()

,count
NEIGHBOURHOOD,
Central Business District,96171
West End,36954
Fairview,29377
Mount Pleasant,26879
Grandview-Woodland,24333
Kitsilano,24118
Renfrew-Collingwood,24092
Kensington-Cedar Cottage,22708
Strathcona,18281


In [5]:
# Display the value counts of the 'HUNDRED_BLOCK' column in the training DataFrame.
# This shows the frequency of specific hundred-block locations, which can indicate high-crime areas.
train_df['HUNDRED_BLOCK'].value_counts()

,count
HUNDRED_BLOCK,
OFFSET TO PROTECT PRIVACY,49365
7XX GRANVILLE ST,4148
6XX GRANVILLE ST,2876
7XX W GEORGIA ST,2141
X NK_LOC ST,2113
...,...
E 54TH AVE / MCKINNON ST,1
VALLEY DR / OLIVER CRES,1
E 35TH AVE / QUEBEC ST,1


In [ ]:
# Load the test dataset from a CSV file using the path defined earlier as `path_2`.
test_df = pd.read_csv(path_2)
# Display the first few rows of the loaded test DataFrame to inspect its structure.
test_df.head()

,YEAR,MONTH,TYPE,Incident_Counts
0,2013,6,Vehicle Collision or Pedestrian Struck (with I...,NaN
1,2013,6,Theft of Vehicle,NaN
2,2013,6,Theft of Bicycle,NaN
3,2013,6,Theft from Vehicle,NaN
4,2013,6,Other Theft,NaN


In [ ]:
test_df_1 = test_df.copy()
# Drop the 'TYPE' column as it's not needed for grouping by year and month.
test_df_1 = test_df_1.drop('TYPE', axis=1)
# Drop the 'Incident_Counts' column as it contains NaN values and is not used for this aggregation.
test_df_1 = test_df_1.drop('Incident_Counts', axis=1)
# Group by 'YEAR' and 'MONTH' to get a unique list of year-month combinations, then count occurrences.
# Rename the count column to 'Counts' and reset the index to make 'YEAR' and 'MONTH' regular columns.
test_df_1 = test_df_1.groupby(['YEAR', 'MONTH'])['MONTH'].count().rename('Counts').reset_index()
# Drop the 'Counts' column as only the unique year-month combinations are needed for forecasting.
test_df_1 = test_df_1.drop('Counts', axis=1)
# Display the resulting DataFrame.
test_df_1

,YEAR,MONTH
0,2012,1
1,2012,2
2,2012,3
3,2012,4
4,2012,5
5,2012,6
6,2012,7
7,2012,8
8,2012,9
9,2012,10


In [ ]:
# Create a new DataFrame `new_rows_df` containing 'YEAR' and 'MONTH' data for the latter half of 2013 (July to December).
# This DataFrame will be used to extend the time series in the test set for future forecasting.
new_rows_df = pd.DataFrame({'YEAR': [2013, 2013, 2013, 2013, 2013, 2013], 'MONTH': [7, 8, 9, 10, 11, 12]})
# Display the newly created DataFrame.
new_rows_df

,YEAR,MONTH
0,2013,7
1,2013,8
2,2013,9
3,2013,10
4,2013,11
5,2013,12


In [ ]:
# Concatenate `test_df_1` (which contains year-month combinations up to June 2013) with `new_rows_df`.
# This extends the test time series to include all months of 2013.
pd.concat([test_df_1, new_rows_df], ignore_index=True)

,YEAR,MONTH
0,2012,1
1,2012,2
2,2012,3
3,2012,4
4,2012,5
5,2012,6
6,2012,7
7,2012,8
8,2012,9
9,2012,10


In [ ]:
# Select 'TYPE' and 'MONTH' columns from the original `test_df` and remove duplicate rows.
# This creates a unique mapping of crime types to months present in the test set.
crime_type_df_1 = test_df[['TYPE', 'MONTH']].drop_duplicates()
# Sort the DataFrame by 'TYPE' and 'MONTH' for better readability and consistent processing.
crime_type_df_1.sort_values(by=['TYPE', 'MONTH'], inplace=True)
# Display the resulting DataFrame.
crime_type_df_1

,TYPE,MONTH
53,Break and Enter Commercial,1
44,Break and Enter Commercial,2
35,Break and Enter Commercial,3
26,Break and Enter Commercial,4
17,Break and Enter Commercial,5
...,...,...
90,Vehicle Collision or Pedestrian Struck (with I...,8
81,Vehicle Collision or Pedestrian Struck (with I...,9
72,Vehicle Collision or Pedestrian Struck (with I...,10
63,Vehicle Collision or Pedestrian Struck (with I...,11


In [ ]:
# Merge `crime_type_df_1` and `test_df_1` on the 'MONTH' column using an outer join.
# This combines all unique crime types and months with the extended year-month combinations.
Test_df2 = pd.merge(crime_type_df_1, test_df_1, on='MONTH', how='outer')
# Sort the merged DataFrame by 'TYPE', 'YEAR', and 'MONTH' for a chronological and type-wise order.
Test_df2.sort_values(by=['TYPE', 'YEAR', 'MONTH'], inplace=True)
# Extract the 'YEAR' column, remove it from its current position, and insert it as the second column.
# This reorders the columns to have 'TYPE', 'YEAR', 'MONTH'.
col_to_move = Test_df2.pop('YEAR')
Test_df2.insert(1, 'YEAR', col_to_move)
# Check for any null values in the final `Test_df2` DataFrame.
Test_df2.isnull().sum()

,0
TYPE,0
YEAR,0
MONTH,0


In [ ]:
# Display the processed `Test_df2` DataFrame.
# This DataFrame now contains a complete grid of all crime types for all months in the test period (2012-2013).
Test_df2

,TYPE,YEAR,MONTH
0,Break and Enter Commercial,2012,1
18,Break and Enter Commercial,2012,2
36,Break and Enter Commercial,2012,3
54,Break and Enter Commercial,2012,4
72,Break and Enter Commercial,2012,5
...,...,...,...
35,Vehicle Collision or Pedestrian Struck (with I...,2013,2
53,Vehicle Collision or Pedestrian Struck (with I...,2013,3
71,Vehicle Collision or Pedestrian Struck (with I...,2013,4
89,Vehicle Collision or Pedestrian Struck (with I...,2013,5


# ***Cleaning the Training Data, Performing EDA & Buildingg Visualizations***

In [6]:
# Display the entire `train_df` DataFrame to get an overview of its contents.
# This is a common step after initial loading and before extensive cleaning or EDA.
train_df

,TYPE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y,Latitude,Longitude,HOUR,MINUTE,YEAR,MONTH,DAY,Date
0,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,16.0,15.0,1999,5,12,1999-05-12
1,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,15.0,20.0,1999,5,7,1999-05-07
2,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,16.0,40.0,1999,4,23,1999-04-23
3,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,11.0,15.0,1999,4,20,1999-04-20
4,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,17.0,45.0,1999,4,12,1999-04-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
474560,Theft from Vehicle,E 28TH AVE / FRASER ST,Riley Park,493434.25,5454756.06,49.245542,-123.090211,19.0,0.0,2011,10,29,2011-10-29
474561,Theft from Vehicle,E 28TH AVE / FRASER ST,Riley Park,493434.25,5454756.06,49.245542,-123.090211,23.0,0.0,2011,3,9,2011-03-09
474562,Other Theft,23XX CAMBIE ST,Mount Pleasant,491656.42,5456901.55,49.264820,-123.114682,17.0,52.0,2011,2,10,2011-02-10
474563,Other Theft,7XX BUTE ST,West End,490797.66,5459282.31,49.286223,-123.126541,18.0,0.0,2011,12,20,2011-12-20


In [ ]:
# Save the `train_df` DataFrame to a CSV file named 'train_df.csv' without the index.
train_df.to_csv('train_df.csv', index=False)
# Import the `files` module from `google.colab` to enable file download functionality.
from google.colab import files
# Initiate the download of the 'train_df.csv' file to the local machine.
files.download('train_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
# Create a copy of the `train_df` DataFrame and assign it to `Train_df`.
# This is good practice to avoid modifying the original DataFrame directly during data cleaning or preprocessing.
Train_df = train_df.copy()
# Display the first few rows of the copied DataFrame.
Train_df

,TYPE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y,Latitude,Longitude,HOUR,MINUTE,YEAR,MONTH,DAY,Date
0,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,16.0,15.0,1999,5,12,1999-05-12
1,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,15.0,20.0,1999,5,7,1999-05-07
2,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,16.0,40.0,1999,4,23,1999-04-23
3,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,11.0,15.0,1999,4,20,1999-04-20
4,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,17.0,45.0,1999,4,12,1999-04-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
474560,Theft from Vehicle,E 28TH AVE / FRASER ST,Riley Park,493434.25,5454756.06,49.245542,-123.090211,19.0,0.0,2011,10,29,2011-10-29
474561,Theft from Vehicle,E 28TH AVE / FRASER ST,Riley Park,493434.25,5454756.06,49.245542,-123.090211,23.0,0.0,2011,3,9,2011-03-09
474562,Other Theft,23XX CAMBIE ST,Mount Pleasant,491656.42,5456901.55,49.264820,-123.114682,17.0,52.0,2011,2,10,2011-02-10
474563,Other Theft,7XX BUTE ST,West End,490797.66,5459282.31,49.286223,-123.126541,18.0,0.0,2011,12,20,2011-12-20


In [8]:
# Calculate and display the sum of null values for each column in the `Train_df` DataFrame.
# This helps in identifying columns with missing data that need to be addressed.
Train_df.isnull().sum()

,0
TYPE,0
HUNDRED_BLOCK,13
NEIGHBOURHOOD,51491
X,0
Y,0
Latitude,0
Longitude,0
HOUR,49365
MINUTE,49365
YEAR,0


In [9]:
# Display a concise summary of the `Train_df` DataFrame.
# This includes information about the index dtype, column dtypes, non-null values, and memory usage.
Train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474565 entries, 0 to 474564
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   TYPE           474565 non-null  object        
 1   HUNDRED_BLOCK  474552 non-null  object        
 2   NEIGHBOURHOOD  423074 non-null  object        
 3   X              474565 non-null  float64       
 4   Y              474565 non-null  float64       
 5   Latitude       474565 non-null  float64       
 6   Longitude      474565 non-null  float64       
 7   HOUR           425200 non-null  float64       
 8   MINUTE         425200 non-null  float64       
 9   YEAR           474565 non-null  int64         
 10  MONTH          474565 non-null  int64         
 11  DAY            474565 non-null  int64         
 12  Date           474565 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(6), int64(3), object(3)
memory usage: 47.1+ MB


In [10]:
# Filter and display rows where the 'NEIGHBOURHOOD' column has null values.
# This allows for a direct inspection of the characteristics of records with missing neighborhood information.
Train_df[Train_df['NEIGHBOURHOOD'].isnull()]

,TYPE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y,Latitude,Longitude,HOUR,MINUTE,YEAR,MONTH,DAY,Date
18,Offence Against a Person,OFFSET TO PROTECT PRIVACY,NaN,0.00,0.00,0.000000,0.000000,NaN,NaN,1999,2,12,1999-02-12
23,Offence Against a Person,OFFSET TO PROTECT PRIVACY,NaN,0.00,0.00,0.000000,0.000000,NaN,NaN,1999,4,4,1999-04-04
31,Offence Against a Person,OFFSET TO PROTECT PRIVACY,NaN,0.00,0.00,0.000000,0.000000,NaN,NaN,1999,3,20,1999-03-20
32,Offence Against a Person,OFFSET TO PROTECT PRIVACY,NaN,0.00,0.00,0.000000,0.000000,NaN,NaN,1999,4,17,1999-04-17
33,Offence Against a Person,OFFSET TO PROTECT PRIVACY,NaN,0.00,0.00,0.000000,0.000000,NaN,NaN,1999,7,14,1999-07-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
472830,Theft of Bicycle,X NK_LOC ST,NaN,492757.49,5458792.47,49.281843,-123.099582,9.0,0.0,2011,9,4,2011-09-04
472832,Theft of Bicycle,X NK_LOC ST,NaN,492757.49,5458792.47,49.281843,-123.099582,9.0,0.0,2011,9,10,2011-09-10
472837,Theft of Bicycle,X NK_LOC ST,NaN,492757.49,5458792.47,49.281843,-123.099582,19.0,2.0,2011,10,1,2011-10-01
472838,Theft of Bicycle,X NK_LOC ST,NaN,492757.49,5458792.47,49.281843,-123.099582,9.0,52.0,2011,10,4,2011-10-04


In [11]:
# Filter and display rows where the 'HUNDRED_BLOCK' column has the value 'OFFSET TO PROTECT PRIVACY'.
# This often indicates anonymized or sensitive location data.
Train_df[Train_df['HUNDRED_BLOCK'] == 'OFFSET TO PROTECT PRIVACY']

,TYPE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y,Latitude,Longitude,HOUR,MINUTE,YEAR,MONTH,DAY,Date
18,Offence Against a Person,OFFSET TO PROTECT PRIVACY,NaN,0.0,0.0,0.0,0.0,NaN,NaN,1999,2,12,1999-02-12
23,Offence Against a Person,OFFSET TO PROTECT PRIVACY,NaN,0.0,0.0,0.0,0.0,NaN,NaN,1999,4,4,1999-04-04
31,Offence Against a Person,OFFSET TO PROTECT PRIVACY,NaN,0.0,0.0,0.0,0.0,NaN,NaN,1999,3,20,1999-03-20
32,Offence Against a Person,OFFSET TO PROTECT PRIVACY,NaN,0.0,0.0,0.0,0.0,NaN,NaN,1999,4,17,1999-04-17
33,Offence Against a Person,OFFSET TO PROTECT PRIVACY,NaN,0.0,0.0,0.0,0.0,NaN,NaN,1999,7,14,1999-07-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
466300,Offence Against a Person,OFFSET TO PROTECT PRIVACY,NaN,0.0,0.0,0.0,0.0,NaN,NaN,2011,3,13,2011-03-13
466308,Offence Against a Person,OFFSET TO PROTECT PRIVACY,NaN,0.0,0.0,0.0,0.0,NaN,NaN,2011,9,14,2011-09-14
466309,Offence Against a Person,OFFSET TO PROTECT PRIVACY,NaN,0.0,0.0,0.0,0.0,NaN,NaN,2011,8,17,2011-08-17
466317,Offence Against a Person,OFFSET TO PROTECT PRIVACY,NaN,0.0,0.0,0.0,0.0,NaN,NaN,2011,4,21,2011-04-21


In [12]:
# Calculate and display the value counts for the 'HUNDRED_BLOCK' column in the training DataFrame.
# This shows the frequency of each hundred block, helping identify high-crime areas or data anomalies like 'OFFSET TO PROTECT PRIVACY'.
Train_df['HUNDRED_BLOCK'].value_counts()

,count
HUNDRED_BLOCK,
OFFSET TO PROTECT PRIVACY,49365
7XX GRANVILLE ST,4148
6XX GRANVILLE ST,2876
7XX W GEORGIA ST,2141
X NK_LOC ST,2113
...,...
E 54TH AVE / MCKINNON ST,1
VALLEY DR / OLIVER CRES,1
E 35TH AVE / QUEBEC ST,1


In [13]:
# Fill any missing (NaN) values in the 'NEIGHBOURHOOD' column with the string 'OFFSET TO PROTECT PRIVACY'.
# This harmonizes the missing neighborhood data with the anonymized hundred block data.
Train_df['NEIGHBOURHOOD'].fillna('OFFSET TO PROTECT PRIVACY', inplace=True)
# Fill any missing (NaN) values in the 'HUNDRED_BLOCK' column with the string 'OFFSET TO PROTECT PRIVACY'.
# This step ensures consistency in handling anonymized or missing location details.
Train_df['HUNDRED_BLOCK'].fillna('OFFSET TO PROTECT PRIVACY', inplace=True)
# Display the updated `Train_df` DataFrame after filling null values.
Train_df

,TYPE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y,Latitude,Longitude,HOUR,MINUTE,YEAR,MONTH,DAY,Date
0,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,16.0,15.0,1999,5,12,1999-05-12
1,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,15.0,20.0,1999,5,7,1999-05-07
2,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,16.0,40.0,1999,4,23,1999-04-23
3,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,11.0,15.0,1999,4,20,1999-04-20
4,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,17.0,45.0,1999,4,12,1999-04-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
474560,Theft from Vehicle,E 28TH AVE / FRASER ST,Riley Park,493434.25,5454756.06,49.245542,-123.090211,19.0,0.0,2011,10,29,2011-10-29
474561,Theft from Vehicle,E 28TH AVE / FRASER ST,Riley Park,493434.25,5454756.06,49.245542,-123.090211,23.0,0.0,2011,3,9,2011-03-09
474562,Other Theft,23XX CAMBIE ST,Mount Pleasant,491656.42,5456901.55,49.264820,-123.114682,17.0,52.0,2011,2,10,2011-02-10
474563,Other Theft,7XX BUTE ST,West End,490797.66,5459282.31,49.286223,-123.126541,18.0,0.0,2011,12,20,2011-12-20


In [14]:
Train_df['Month_name'] = np.nan

# Define a function to convert numerical month values (1-12) to their corresponding names.
def get_month_name(month_num):
  if month_num == 1:
    return 'January'
  elif month_num == 2:
    return 'February'
  elif month_num == 3:
    return 'March'
  elif month_num == 4:
    return 'April'
  elif month_num == 5:
    return 'May'
  elif month_num == 6:
    return 'June'
  elif month_num == 7:
    return 'July'
  elif month_num == 8:
    return 'August'
  elif month_num == 9:
    return 'September'
  elif month_num == 10:
    return 'October'
  elif month_num == 11:
    return 'November'
  else:
    return 'December'

# Apply the function to the 'MONTH' column to populate the new 'Month_name' column.
Train_df['Month_name'] = Train_df['MONTH'].apply(get_month_name)

# Display the 'MONTH' and newly created 'Month_name' columns to verify the conversion.
Train_df[['MONTH', 'Month_name']]

,MONTH,Month_name
0,5,May
1,5,May
2,4,April
3,4,April
4,4,April
...,...,...
474560,10,October
474561,3,March
474562,2,February
474563,12,December


# ***Building Visualizations to Uncover Trends and Patterns from Historical Data***

# *Crime Count Evaluation on the Basis of Crime Type*

In [ ]:
# Group the `Train_df` by 'YEAR' and count the occurrences of 'TYPE' (representing crime counts).
yearly_crime_count = Train_df.groupby('YEAR')['TYPE'].count()
# Display the yearly crime counts.
yearly_crime_count

# Create a line plot using Plotly Express to visualize the yearly crime count.
fig = px.line(x=yearly_crime_count.index, y=yearly_crime_count.values, title='Yearly Crime Count')
# Update the layout of the plot for better aesthetics and readability.
fig.update_layout(xaxis_title='Year', yaxis_title='Crime Count', title_x = 0.5, title_font_size=24, title_font_family='Arial Black', width=1200, height=500)
# Display the plot.
fig.show()

***Trend Observed:***
* The total number of crimes varies significantly across
years.
* Crime does not follow a constant trend but shows periods  of increase and decrease.

***Key Insight:***

Crime trends show clear temporal variability, indicating that crime occurrence is influenced by long-term structural and socio-economic factors rather than random variation.

***Pattern Interpretation:***
*   Some years show noticeable increases in crime counts compared to previous years.
*   Other years show stabilization or slight decrease.

***Conclusion from Visualization:***

Crime trends evolve gradually over time and exhibit predictable patterns suitable for forecasting.

In [ ]:
import plotly.express as px

# Group the training DataFrame by 'YEAR' and 'TYPE' to count the occurrences of each crime type per year.
yearly_crime_type_count = Train_df.groupby(['YEAR', 'TYPE']).size().reset_index(name='Count')

# Sort the results for better visualization and consistency.
yearly_crime_type_count.sort_values(by=['YEAR', 'TYPE'], ascending=[True, True], inplace=True)
yearly_crime_type_count.reset_index(drop=True, inplace=True)

# Create a stacked bar chart using Plotly Express to visualize yearly crime counts by type.
# Each bar represents a year, segmented by different crime types.
fig = px.bar(yearly_crime_type_count, x='YEAR', y='Count', color='TYPE', title='Yearly Crime Count by Type',
             hover_data={'YEAR': True, 'TYPE': True, 'Count': True}) # Add hover details for interactivity.

# Update the layout of the plot for better readability and aesthetics.
fig.update_layout(xaxis_title='Year', yaxis_title='Crime Count', title_x=0.5, title_font_size=24, title_font_family='Arial Black', width=1200, height=500)

# Ensure the x-axis ticks are integers, representing each year clearly.
fig.update_xaxes(dtick=1)

# Display the plot.
fig.show()

***Trend Observed:***
*   Certain crime types consistently contribute more to total crime volume.
*   Some crime types remain dominant across all years.

***Key Insight:***

Crime distribution is uneven across crime types, with a few crime categories contributing disproportionately to total crime.

***Pattern Interpretation:***
*   High-frequency crime types remain consistently high across years.
*   Low-frequency crimes remain relatively stable.

***Conclusion:***

Crime prevention strategies should prioritize dominant crime types to achieve maximum crime reduction.

In [ ]:

# Group the `Train_df` by 'YEAR', 'MONTH', and 'Month_name' and count the number of 'TYPE' entries to get monthly crime counts.
monthly_crime_count = Train_df.groupby(['YEAR', 'MONTH', 'Month_name'])['TYPE'].count()
# Convert the grouped series to a DataFrame and reset the index.
monthly_crime_count_df = monthly_crime_count.reset_index()
# Rename the 'TYPE' column to 'Crime Count' for clarity.
monthly_crime_count_df.columns = ['YEAR', 'MONTH', 'Month_name', 'TYPE']
# Sort the DataFrame by year, month, and month name for chronological order.
monthly_crime_count_df = monthly_crime_count_df.sort_values(by=['YEAR', 'MONTH', 'Month_name'], ascending=True)
# Reset the index of the DataFrame.
monthly_crime_count_df.reset_index(drop=True)


# Create a line plot using Plotly Express to visualize the monthly crime count over the years.
# Each line represents a different year, showing the trend of crime counts across months.
fig = px.line(monthly_crime_count_df, x='Month_name', y='TYPE', title='Monthly Crime Count by Year', color='YEAR', labels={'TYPE': 'Crime Count'})
# Update the layout of the plot for better aesthetics and readability.
fig.update_layout(xaxis_title='Month', yaxis_title='Crime Count', title_x = 0.5, title_font_size=24, title_font_family='Arial Black', width=1000, height=700)
# Display the plot.
fig.show()

***Trend Observed:***
* Crime counts fluctuate significantly across months within each year.
* Clear seasonal peaks and troughs are visible.

***Key Insight:***

Crime occurrence follows strong seasonal patterns.

***Pattern Interpretation:***
*   Crime tends to increase during certain months consistently across multiple years.
*   Crime decreases during certain months consistently.

***Conclusion:***

Crime forecasting models must incorporate seasonal components.

In [ ]:

# Group the `Train_df` by 'YEAR', 'MONTH', 'Month_name', and 'TYPE' to count crime occurrences by type per month.
monthly_crime_type_count = Train_df.groupby(['YEAR', 'MONTH', 'Month_name', 'TYPE']).size().reset_index(name='Count')
# Rename the columns for clarity.
monthly_crime_type_count.columns = ['YEAR', 'MONTH', 'Month_name', 'TYPE', 'Count']
# Sort the DataFrame chronologically.
monthly_crime_type_count = monthly_crime_type_count.sort_values(by=['YEAR', 'MONTH'], ascending=True)
# Reset the index.
monthly_crime_type_count.reset_index(drop=True)

# Create a stacked bar chart using Plotly Express to visualize monthly crime counts by type.
# Each bar represents a month, segmented by different crime types, with hover details for interactivity.
fig = px.bar(monthly_crime_type_count, x='Month_name', y='Count', color='TYPE', title='Monthly Crime Count by Type',
             hover_data={'YEAR': True, 'Month_name': True, 'TYPE': True, 'Count': True})
# Update the layout of the plot for better readability and aesthetics.
fig.update_layout(xaxis_title='Month', yaxis_title='Crime Count', title_x=0.5, title_font_size=24, title_font_family='Arial Black', width=1000, height=600)
# Display the plot.
fig.show()

***Trend Observed:***
* Different crime types show distinct monthly patterns.
* Some crime types peak during specific months.

***Key Insight:***

Crime type occurrence varies seasonally.

***Pattern Interpretation:***

Certain crime types are more sensitive to seasonal changes.

In [ ]:

# Group the `Train_df` by 'YEAR', 'MONTH', and 'Month_name' to get the total crime count for each month and year.
df = Train_df.groupby(['YEAR', 'MONTH', 'Month_name']).size().reset_index(name='Crime_Count')
# Rename columns for clarity.
df.columns = ['YEAR', 'MONTH', 'Month_name', 'Crime_Count']
# Sort the DataFrame chronologically.
df = df.sort_values(by=['YEAR', 'MONTH', 'Month_name'], ascending=True)
# Reset the index.
df.reset_index(drop=True, inplace=True)

# Create a density heatmap using Plotly Express to visualize crime counts over months and years.
# The color intensity represents the crime count, allowing easy identification of patterns.
fig = px.density_heatmap(df, x='Month_name', y='YEAR', z='Crime_Count', histfunc='sum',
             title='Crime Count over the years',
             labels={'Month_name': 'Month', 'YEAR': 'Year', 'Crime_Count': 'Crime Count'})
# Update the layout for better readability and aesthetics.
fig.update_layout(xaxis_title='Month', yaxis_title='Year', title_x=0.5, title_font_size=24, title_font_family='Arial Black', width=1000, height=600)
# Set the y-axis type to 'category' to ensure years are treated as distinct categories.
fig.update_yaxes(type='category')
# Customize hover information to show Month, Year, and Crime Count.
fig.update_traces(hovertemplate='Month: %{x}<br>Year: %{y}<br>Crime Count: %{z}', selector=dict(type='heatmap'))
# Display the plot.
fig.show()

***Trend Observed:***

High-intensity regions indicate months and years with high crime activity.

***Key Insight:***

Certain month-year combinations consistently show higher crime density.

***Conclusion:***

Crime patterns exhibit structured temporal concentration.

In [29]:

# Group the `Train_df` by 'YEAR', 'MONTH', and 'Month_name' to get the total crime count for each period.
df = Train_df.groupby(['YEAR', 'MONTH', 'Month_name']).size().reset_index(name='Crime_Count')
# Rename columns for consistency.
df.columns = ['YEAR', 'MONTH', 'Month_name', 'Crime_Count']
# Sort the DataFrame chronologically.
df = df.sort_values(by=['YEAR', 'MONTH'], ascending=True)
# Reset the index.
df.reset_index(drop=True, inplace=True)

# Create a 3D scatter plot using Plotly Express to visualize crime count across Year, Month, and Crime_Count.
# 'color' and 'size' are mapped to 'Crime_Count' to add depth to the visualization.
fig = px.scatter_3d(df, x='YEAR', y='Month_name',
                 z='Crime_Count',
                 color='Crime_Count',
                 color_continuous_scale='Viridis',
                 size='Crime_Count',
                 title='Crime Count over the years',
                 labels={'Month_name': 'Month', 'YEAR': 'Year', 'Crime_Count': 'Crime Count'})

# Customize hover information for better interactivity, showing Year, Month, and Crime Count.
fig.update_traces(hovertemplate='<b>Year</b>: %{x}<br><b>Month</b>: %{y}<br><b>Crime Count</b>: %{z}<extra></extra>')

# Update the layout of the 3D plot for better aesthetics and readability.
fig.update_layout(
    scene=dict(xaxis_title='Year', yaxis_title='Month', zaxis_title='Crime Count'),
    width=1000,
    height=800,
    title_x=0.5,  # Center the title
    title_font_color='RebeccaPurple',  # Set title color to deep purple
    title_font_family='Arial Black', # Make title bold
    title_font_size=28, # Make title bigger
    paper_bgcolor='AliceBlue', # Lighter background for the entire figure
    plot_bgcolor='AliceBlue' # Lighter background for the plotting area
)

# Display the plot.
fig.show()

# *Crime Count Evaluation On a Neighbourhood Level*

In [ ]:
# Install necessary libraries: pandas, geopy, and folium.
# This ensures that all required packages for geospatial analysis and mapping are available in the environment.
!pip install pandas geopy folium

In [ ]:
# Display the `Train_df` DataFrame. This provides a quick look at the cleaned and feature-engineered training data before further analysis.
Train_df

,TYPE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y,Latitude,Longitude,HOUR,MINUTE,YEAR,MONTH,DAY,Date,Month_name
0,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,16.0,15.0,1999,5,12,1999-05-12,May
1,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,15.0,20.0,1999,5,7,1999-05-07,May
2,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,16.0,40.0,1999,4,23,1999-04-23,April
3,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,11.0,15.0,1999,4,20,1999-04-20,April
4,Other Theft,9XX TERMINAL AVE,Strathcona,493906.50,5457452.47,49.269802,-123.083763,17.0,45.0,1999,4,12,1999-04-12,April
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474560,Theft from Vehicle,E 28TH AVE / FRASER ST,Riley Park,493434.25,5454756.06,49.245542,-123.090211,19.0,0.0,2011,10,29,2011-10-29,October
474561,Theft from Vehicle,E 28TH AVE / FRASER ST,Riley Park,493434.25,5454756.06,49.245542,-123.090211,23.0,0.0,2011,3,9,2011-03-09,March
474562,Other Theft,23XX CAMBIE ST,Mount Pleasant,491656.42,5456901.55,49.264820,-123.114682,17.0,52.0,2011,2,10,2011-02-10,February
474563,Other Theft,7XX BUTE ST,West End,490797.66,5459282.31,49.286223,-123.126541,18.0,0.0,2011,12,20,2011-12-20,December


In [ ]:
# Create a new DataFrame `df` by selecting specific columns ('NEIGHBOURHOOD', 'HUNDRED_BLOCK', 'Latitude', 'Longitude') from `Train_df`.
# This subset is used for geographical visualization.
df = Train_df[['NEIGHBOURHOOD', 'HUNDRED_BLOCK', 'Latitude', 'Longitude']].copy()
# Check for any null values in the newly created `df`.
# This confirms that all relevant geographical columns are filled after earlier cleaning steps.
df.isnull().sum()

,0
NEIGHBOURHOOD,0
HUNDRED_BLOCK,0
Latitude,0
Longitude,0


In [ ]:
import folium

# Group the training data by 'NEIGHBOURHOOD' and 'HUNDRED_BLOCK' and count the 'TYPE' of crimes.
# This aggregates crime counts at a granular geographical level.
neighbourhood_crime_count = train_df.groupby(['NEIGHBOURHOOD', 'HUNDRED_BLOCK'])['TYPE'].count().reset_index(name='Crime_Count')
# Fill any missing 'HUNDRED_BLOCK' values with 'OFFSET TO PROTECT PRIVACY', consistent with previous cleaning.
neighbourhood_crime_count['HUNDRED_BLOCK'].fillna('OFFSET TO PROTECT PRIVACY', inplace=True)
# Calculate the mean Latitude and Longitude for each 'NEIGHBOURHOOD'.
# This provides a central coordinate for each neighborhood.
Latitude_Longitude_df = train_df.groupby('NEIGHBOURHOOD').agg({'Latitude': 'mean', 'Longitude': 'mean'}).reset_index()
# Merge the crime counts with the mean latitude and longitude data based on 'NEIGHBOURHOOD'.
neighbourhood_crime_count_df = pd.merge(neighbourhood_crime_count, Latitude_Longitude_df, on='NEIGHBOURHOOD')
# Sort the DataFrame for consistent processing and display.
neighbourhood_crime_count_df.sort_values(by=['NEIGHBOURHOOD', 'HUNDRED_BLOCK'], inplace=True)


# Initialize a Folium map, centered at the mean latitude and longitude of all crime incidents.
crime_map = folium.Map(location=[neighbourhood_crime_count_df['Latitude'].mean(), neighbourhood_crime_count_df['Longitude'].mean()], zoom_start=12)


# Define a dictionary for crime type colors (though not directly used in this specific CircleMarker loop, it was likely intended for a different visualization).
crime_colours = {"Break and Enter Commercial":"red",
                 "Break and Enter Residential/Other":"blue",
                 "Mischief":"green",
                 "Offence Against a Person":"purple",
                 "Other Theft":"orange",
                 "Theft from Vehicle":"pink",
                 "Theft of Bicycle":"brown",
                 "Theft of Vehicle":"gray",
                 "Vehicle Collision or Pedestrian Struck (with Injury)":"olive"}

# Adding crime points as CircleMarkers to the map.
# Each marker's size is proportional to the 'Crime_Count' at that location, making denser crime areas visually larger.
for index, row in neighbourhood_crime_count_df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=row['Crime_Count'] * 0.001, # Scale radius based on crime count
        popup=f"{row['NEIGHBOURHOOD']} - {row['HUNDRED_BLOCK']}: {row['Crime_Count']} crimes", # Popup with detailed info
        color= 'red', # Outline color of the circle
        fill=True,
        fill_color='red',
        fill_opacity=0.6 # Opacity of the fill color
    ).add_to(crime_map)

# Display the generated Folium map.
crime_map

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Recalculate and display the sum of null values for each column in the `Train_df` DataFrame.
# This check confirms the current state of missing values after any previous cleaning steps.
Train_df.isnull().sum()

,0
TYPE,0
HUNDRED_BLOCK,0
NEIGHBOURHOOD,0
X,0
Y,0
Latitude,0
Longitude,0
HOUR,49365
MINUTE,49365
YEAR,0


In [ ]:
# Group `Train_df` by 'NEIGHBOURHOOD' and 'TYPE' to count the occurrences of each crime type within each neighborhood.
neighbourhood_type_count = Train_df.groupby(['NEIGHBOURHOOD', 'TYPE']).size().reset_index(name='Count')
# Rename the 'Count' column to 'Crime_Count' for clarity.
neighbourhood_type_count.rename(columns={'Count': 'Crime_Count'}, inplace=True)
# Sort the DataFrame by 'NEIGHBOURHOOD' and 'TYPE' for ordered display.
neighbourhood_type_count.sort_values(by=['NEIGHBOURHOOD', 'TYPE'], inplace=True)

# Create a density heatmap using Plotly Express to visualize crime counts by neighborhood and type.
# The intensity of the color indicates the 'Crime_Count' for a specific neighborhood and crime type.
fig = px.density_heatmap(neighbourhood_type_count, x='NEIGHBOURHOOD', y='TYPE', z='Crime_Count',
             title='Crime Count by Neighbourhood and Type',
             labels={'NEIGHBOURHOOD': 'Neighbourhood', 'TYPE': 'Crime Type', 'Crime_Count': 'Crime Count'})
# Update the layout for better readability, adjusting title position, font, and size.
fig.update_layout(xaxis_title='Neighbourhood', yaxis_title='Crime Type', title_x=0.5, title_font_size=24, title_font_family='Arial Black', width=1200, height=800)
# Adjust the font size of x and y axis titles.
fig.update_xaxes(title_font_size=14)
fig.update_yaxes(title_font_size=14)
# Display the plot.
fig.show()

***Trend Observed:***

Crime distribution varies significantly across neighbourhoods.

***Key Insight:***

Certain neighbourhoods consistently experience higher crime levels.

***Pattern Interpretation:***

Crime is geographically concentrated.

***Conclusion:***

Targeted neighbourhood-level intervention is necessary.

In [ ]:
# Group `Train_df` by 'NEIGHBOURHOOD', 'TYPE', and 'YEAR' to count crime occurrences annually per type and neighborhood.
df = Train_df.groupby(['NEIGHBOURHOOD', 'TYPE', 'YEAR']).size().reset_index(name='Yearly_Crime_Count_Type_wise')
# Sort the DataFrame chronologically and by neighborhood and crime type.
df.sort_values(by=['NEIGHBOURHOOD', 'TYPE', 'YEAR'], ascending=[True, True, True], inplace=True)
# Reset the index.
df.reset_index(drop=True, inplace=True)
# Display the resulting DataFrame.
df


# Create a density heatmap with facets using Plotly Express to visualize yearly crime counts by neighborhood and type.
# 'facet_col='YEAR'' creates separate subplots for each year, allowing comparison across time.
fig = px.density_heatmap(df, x='NEIGHBOURHOOD', y='TYPE', z='Yearly_Crime_Count_Type_wise', facet_col='YEAR', facet_col_wrap=4, histfunc='sum',
             title='Yearly Crime Count by Neighbourhood and Type',
             labels={'NEIGHBOURHOOD': 'Neighbourhood', 'TYPE': 'Crime Type', 'Yearly_Crime_Count_Type_wise': 'Crime Count'})
# Update the layout for better readability, adjusting title position, font, and size, and overall plot dimensions.
fig.update_layout(xaxis_title='Neighbourhood', yaxis_title='Crime Type', title_x=0.5, title_font_size=24, title_font_family='Arial Black', width=1600, height=1200)
# Display the plot.
fig.show()

In [ ]:
# Group `Train_df` by 'NEIGHBOURHOOD' and 'YEAR' to get the total crime count for each neighborhood per year.
Yearly_neighbourhood_crime_count = Train_df.groupby(['NEIGHBOURHOOD', 'YEAR']).size().reset_index(name='Crime Count')
# Sort the DataFrame by neighborhood and year for chronological and grouped display.
Yearly_neighbourhood_crime_count.sort_values(by=['NEIGHBOURHOOD', 'YEAR'], ascending=[True, True], inplace=True)
# Reset the index.
Yearly_neighbourhood_crime_count.reset_index(drop=True, inplace=True)

# Create a line plot using Plotly Express to visualize crime counts over the years for each neighborhood.
# Each line represents a different neighborhood, showing its individual crime trend.
fig = px.line(Yearly_neighbourhood_crime_count, x='YEAR', y='Crime Count', title='Crime Count Over the Years per Neighbourhood', color='NEIGHBOURHOOD', labels={'YEAR': 'Year'})
# Update the layout for better aesthetics, including title position, font, and plot dimensions.
fig.update_layout(xaxis_title='Year', yaxis_title='Crime Count', title_x = 0.5, title_font_size=24, title_font_family='Arial Black', width=1200, height=700)
# Display the plot.
fig.show()


***Trend Observed:***

Some neighbourhoods consistently show higher crime counts than others.

***Key Insight:***

Crime hotspots persist across years.

***Pattern Interpretation:***

High-crime neighbourhoods remain high-crime areas over time.

In [ ]:

# Group `Train_df` by 'NEIGHBOURHOOD', 'MONTH', and 'Month_name' to get crime counts per month for each neighborhood.
Train_df_4 = Train_df.groupby(['NEIGHBOURHOOD', 'MONTH', 'Month_name']).size().reset_index(name='Crime Count')
# Rename columns for clarity.
Train_df_4.columns = ['NEIGHBOURHOOD', 'MONTH', 'Month_name', 'Crime Count']
# Sort the DataFrame chronologically and by neighborhood.
Train_df_4.sort_values(by=['NEIGHBOURHOOD', 'MONTH', 'Month_name'], ascending=[True, True, True], inplace=True)
# Reset the index.
Train_df_4.reset_index(drop=True, inplace=True)

# Create a line plot using Plotly Express to visualize crime count peak months per neighborhood.
# Each line represents a different neighborhood, illustrating monthly crime patterns.
fig = px.line(Train_df_4, x='Month_name', y='Crime Count', title='Crime Count Peak Months Per Neighbourhood', color='NEIGHBOURHOOD', labels={'Month_name': 'Month'})
# Update the layout for better aesthetics, including title position, font, and plot dimensions.
fig.update_layout(xaxis_title='Month', yaxis_title='Crime Count', title_x = 0.5, title_font_size=24, title_font_family='Arial Black', width=1200, height=700)
# Display the plot.
fig.show()

***Trend Observed:***

Each neighbourhood shows specific peak months.

***Key Insight:***

Seasonal crime patterns vary by location.

In [30]:
# Group `Train_df` by 'NEIGHBOURHOOD', 'YEAR', 'MONTH', and 'Month_name' to get monthly crime counts per neighborhood and year.
df = Train_df.groupby(['NEIGHBOURHOOD','YEAR', 'MONTH', 'Month_name']).size().reset_index(name='Monthly_Crime_Count_per_Neighbourhood')
# Sort the DataFrame chronologically and by neighborhood for consistent display.
df.sort_values(by=['NEIGHBOURHOOD', 'YEAR', 'MONTH', 'Month_name'], ascending=[True, True, True, True], inplace=True)
# Reset the index.
df.reset_index(drop=True, inplace=True)


# Create a faceted line plot using Plotly Express to visualize monthly crime counts per neighborhood.
# 'facet_col='NEIGHBOURHOOD'' creates a separate subplot for each neighborhood, and 'color='YEAR'' distinguishes lines by year.
fig = px.line(df, x='Month_name', y='Monthly_Crime_Count_per_Neighbourhood', facet_col='NEIGHBOURHOOD', facet_col_wrap=5, color='YEAR', title='Monthly Crime Count per Neighbourhood',
             labels={'Month_name': 'Month', 'Monthly_Crime_Count_per_Neighbourhood': 'Monthly Crime Count', 'YEAR': 'Year'})
# Update the layout for better aesthetics, adjusting title position, font, and overall plot dimensions.
fig.update_layout(xaxis_title='Month', yaxis_title='Monthly Crime Count', title_x=0.5, title_font_size=24, title_font_family='Arial Black', width=1600, height=1600)
# Display the plot.
fig.show()

***Trend Observed:***

Monthly crime trends differ across neighbourhoods.

***Key Insight:***

Crime patterns are influenced by both time and location.

In [ ]:
# Group `Train_df` by 'TYPE' and 'YEAR' to get the annual crime count for each crime type.
Yearly_type_wise_crime_count = Train_df.groupby(['TYPE', 'YEAR']).size().reset_index(name='Crime Count')
# Sort the DataFrame by crime type and year for chronological and grouped display.
Yearly_type_wise_crime_count.sort_values(by=['TYPE', 'YEAR'], ascending=[True, True], inplace=True)
# Reset the index.
Yearly_type_wise_crime_count.reset_index(drop=True, inplace=True)

# Create a line plot using Plotly Express to visualize the crime count over the years for each type.
# Each line represents a different crime type, showing its trend over time.
fig = px.line(Yearly_type_wise_crime_count, x='YEAR', y='Crime Count', color='TYPE', title='Type Wise Crime Count Over the Years', labels={'YEAR':'Year'})
# Update the layout for better aesthetics, including title position, font, and plot dimensions.
fig.update_layout(xaxis_title='Year', yaxis_title='Crime Count', title_x = 0.5, title_font_size=24, title_font_family='Arial Black', width=1200, height=700)
# Display the plot.
fig.show()

***Trend Observed:***

Each crime type follows its own long-term trend.

***Key Insight:***

Crime evolution differs by type.

In [ ]:
# Group `Train_df` by 'TYPE', 'MONTH', and 'Month_name' to get crime counts per month for each crime type.
df = Train_df.groupby(['TYPE', 'MONTH', 'Month_name']).size().reset_index(name='Crime_Count')
# Sort the DataFrame chronologically and by crime type.
df.sort_values(by=['TYPE', 'MONTH', 'Month_name'], ascending=[True, True, True], inplace=True)
# Reset the index.
df.reset_index(drop=True, inplace=True)

# Create a stacked bar chart using Plotly Express to visualize crime count peak months.
# Each bar segment represents a crime type, showing its contribution to the monthly total.
fig = px.bar(df, x='Month_name', y='Crime_Count', color='TYPE', title='Crime Count Peak Months',
             labels={'MONTH': 'Month', 'Crime_Count': 'Crime Count'})
# Update the layout for better readability, adjusting title position, font, and plot dimensions.
fig.update_layout(xaxis_title='Month', yaxis_title='Crime Count', title_x=0.5, title_font_size=24, title_font_family='Arial Black', width=1200, height=700)
# Display the plot.
fig.show()

In [ ]:
# Group `Train_df` by 'TYPE', 'MONTH', and 'Month_name' to get crime counts per month for each crime type.
df = Train_df.groupby(['TYPE', 'MONTH', 'Month_name']).size().reset_index(name='Crime_Count')
# Sort the DataFrame chronologically and by crime type.
df.sort_values(by=['TYPE', 'MONTH', 'Month_name'], ascending=[True, True, True], inplace=True)
# Reset the index.
df.reset_index(drop=True, inplace=True)

# Create a line plot using Plotly Express to visualize crime count peak months.
# Each line represents a crime type, showing its monthly trend.
fig = px.line(df, x='Month_name', y='Crime_Count', color='TYPE', title='Crime Count Peak Months',
             labels={'MONTH': 'Month', 'Crime_Count': 'Crime Count'})
# Update the layout for better readability, adjusting title position, font, and plot dimensions.
fig.update_layout(xaxis_title='Month', yaxis_title='Crime Count', title_x=0.5, title_font_size=24, title_font_family='Arial Black', width=1200, height=700)
# Display the plot.
fig.show()

***Trend Observed:***

Certain crime types show consistent peak months.

***Key Insight:***

Seasonality exists at crime type level.

In [31]:
# Group `Train_df` by 'TYPE', 'YEAR', 'MONTH', and 'Month_name' to get monthly crime counts per type and year.
df = Train_df.groupby(['TYPE', 'YEAR', 'MONTH', 'Month_name']).size().reset_index(name='Crime_Count')
# Rename columns for clarity and consistency.
df.columns=['TYPE', 'YEAR', 'MONTH', 'Month_name', 'Crime_Count']
# Sort the DataFrame chronologically and by crime type.
df.sort_values(by=['TYPE', 'YEAR', 'MONTH', 'Month_name'], ascending=[True, True, True, True], inplace=True)
# Reset the index.
df.reset_index(drop=True, inplace=True)

# Create a faceted line plot using Plotly Express to visualize crime count peak months by crime type.
# 'facet_col='TYPE'' creates separate subplots for each crime type, and 'color='YEAR'' distinguishes lines by year.
fig = px.line(df, x='Month_name', y='Crime_Count', color='YEAR', title='Crime Count Peak Months', facet_col='TYPE', facet_col_wrap=3,
             labels={'MONTH': 'Month', 'Crime_Count': 'Crime Count'})
# Update the layout for better aesthetics, adjusting title position, font, and overall plot dimensions.
fig.update_layout(xaxis_title='Month', yaxis_title='Crime Count', title_x=0.5, title_font_size=24, title_font_family='Arial Black', width=1200, height=1200)
# Display the plot.
fig.show()

***Trend Observed:***

Seasonal pattern consistency across years.

# ***Preparing the Datasets to be used for Model Training***

In [32]:
# Group the `Train_df` by 'TYPE', 'YEAR', and 'MONTH' to aggregate monthly crime counts for each type.
Train_df_2 = Train_df.groupby(['TYPE', 'YEAR', 'MONTH']).size().reset_index(name='Monthly_Crime_Count_Type_wise')
# Sort the DataFrame chronologically and by crime type for consistent time series analysis.
Train_df_2.sort_values(by=['TYPE', 'YEAR', 'MONTH'], ascending=[True, True, True], inplace=True)
# Reset the index.
Train_df_2.reset_index(drop=True, inplace=True)
# Display the resulting DataFrame.
Train_df_2

,TYPE,YEAR,MONTH,Monthly_Crime_Count_Type_wise
0,Break and Enter Commercial,1999,1,303
1,Break and Enter Commercial,1999,2,254
2,Break and Enter Commercial,1999,3,292
3,Break and Enter Commercial,1999,4,266
4,Break and Enter Commercial,1999,5,291
...,...,...,...,...
1399,Vehicle Collision or Pedestrian Struck (with I...,2011,8,148
1400,Vehicle Collision or Pedestrian Struck (with I...,2011,9,142
1401,Vehicle Collision or Pedestrian Struck (with I...,2011,10,134
1402,Vehicle Collision or Pedestrian Struck (with I...,2011,11,158


In [33]:
# Check for any null values in the `Train_df_2` DataFrame.
# This is a crucial step to ensure the aggregated data is complete before model training.
Train_df_2.isnull().sum()

,0
TYPE,0
YEAR,0
MONTH,0
Monthly_Crime_Count_Type_wise,0


In [34]:
# Filling missing months with zero for a continuous time series

import pandas as pd
import itertools

# 1. Prepare the existing aggregated data
# Train_df_2 should have columns: 'TYPE', 'YEAR', 'MONTH', 'Monthly_Crime_Count_Type_wise'

# 2. Define the full range of time and categories to ensure no gaps
all_crime_types = Train_df_2['TYPE'].unique()
all_years = Train_df_2['YEAR'].unique()
all_months = range(1, 13) # Months 1 to 12

# 3. Create all possible combinations (Cartesian Product) of crime types, years, and months.
# This generates a complete time series framework for every crime type.
all_combinations = list(itertools.product(all_crime_types, all_years, all_months))

# 4. Create a master DataFrame with all generated combinations.
master_df = pd.DataFrame(all_combinations, columns=['TYPE', 'YEAR', 'MONTH'])

# 5. Merge the actual crime data (Train_df_2) into the master DataFrame.
# A left merge ensures all combinations from master_df are kept, filling non-existent actuals with NaN.
df2_continuous = pd.merge(master_df, Train_df_2, on=['TYPE', 'YEAR', 'MONTH'], how='left')

# 6. Fill the gaps (NaN values) in 'Monthly_Crime_Count_Type_wise' with 0 and convert to integer type.
# This assumes that missing data points mean zero crime counts for those periods.
df2_continuous['Monthly_Crime_Count_Type_wise'] = df2_continuous['Monthly_Crime_Count_Type_wise'].fillna(0).astype(int)

# 7. Sort the DataFrame (Crucial for time series analysis and creating lagged features).
# Sorting by 'TYPE', then 'YEAR', then 'MONTH' ensures the time series is in the correct order for each crime type.
df2_continuous = df2_continuous.sort_values(by=['TYPE', 'YEAR', 'MONTH']).reset_index(drop=True)

# Print comparison of row counts and the head of the continuous DataFrame.
print(f'Original rows: {len(Train_df_2)}')
print(f'Continuous rows: {len(df2_continuous)}')
print(df2_continuous)

Original rows: 1404
Continuous rows: 1404
                                                   TYPE  YEAR  MONTH  \
0                            Break and Enter Commercial  1999      1   
1                            Break and Enter Commercial  1999      2   
2                            Break and Enter Commercial  1999      3   
3                            Break and Enter Commercial  1999      4   
4                            Break and Enter Commercial  1999      5   
...                                                 ...   ...    ...   
1399  Vehicle Collision or Pedestrian Struck (with I...  2011      8   
1400  Vehicle Collision or Pedestrian Struck (with I...  2011      9   
1401  Vehicle Collision or Pedestrian Struck (with I...  2011     10   
1402  Vehicle Collision or Pedestrian Struck (with I...  2011     11   
1403  Vehicle Collision or Pedestrian Struck (with I...  2011     12   

      Monthly_Crime_Count_Type_wise  
0                               303  
1                

In [ ]:
# Create a copy of the `Test_df2` DataFrame.
Test_df_2 = Test_df2.copy()
# Add a new column 'Monthly_Crime_Count_Type_wise' and initialize it with NaN values.
# This column will later hold the predicted crime counts for the test period.
Test_df_2['Monthly_Crime_Count_Type_wise'] = np.nan
# Display the first few rows of the modified `Test_df_2`.
Test_df_2

,TYPE,YEAR,MONTH,Monthly_Crime_Count_Type_wise
0,Break and Enter Commercial,2012,1,NaN
18,Break and Enter Commercial,2012,2,NaN
36,Break and Enter Commercial,2012,3,NaN
54,Break and Enter Commercial,2012,4,NaN
72,Break and Enter Commercial,2012,5,NaN
...,...,...,...,...
35,Vehicle Collision or Pedestrian Struck (with I...,2013,2,NaN
53,Vehicle Collision or Pedestrian Struck (with I...,2013,3,NaN
71,Vehicle Collision or Pedestrian Struck (with I...,2013,4,NaN
89,Vehicle Collision or Pedestrian Struck (with I...,2013,5,NaN


In [ ]:
# Save the `Train_df_2` DataFrame to a CSV file named 'Train_df_2.csv' without the index.
Train_df_2.to_csv('Train_df_2.csv', index=False)
# Import the `files` module from `google.colab`.
from google.colab import files
# Initiate the download of the 'Train_df_2.csv' file.
files.download('Train_df_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ***Model Training***

In [35]:
# Install `xgboost`, `sklearn`, and `torchmetrics` libraries.
# These libraries are essential for implementing gradient boosting models, machine learning utilities, and advanced metrics.
!pip install xgboost
!pip install sklearn
!pip install torchmetrics

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 19.7 MB/s eta 0:00:00


In [36]:
import sklearn
from xgboost import XGBRegressor # Import XGBoost Regressor model.
from sklearn.ensemble import RandomForestRegressor # Import RandomForest Regressor model.
from sklearn.ensemble import StackingRegressor # Import Stacking Regressor for ensemble modeling.
from sklearn.linear_model import LinearRegression # Import Linear Regression model.
from sklearn.linear_model import Ridge # Import Ridge Regression model.
from sklearn.metrics import mean_squared_error, r2_score # Import metrics for model evaluation: Mean Squared Error, R-squared.
from sklearn.metrics import mean_absolute_error # Import Mean Absolute Error.
from sklearn.metrics import root_mean_squared_error # Import Root Mean Squared Error.
from sklearn.metrics import mean_absolute_percentage_error # Import Mean Absolute Percentage Error.
import torch # Import PyTorch library, often used for deep learning components or custom metrics.
from torchmetrics import WeightedMeanAbsolutePercentageError # Import Weighted Mean Absolute Percentage Error from torchmetrics.
from sklearn.model_selection import GridSearchCV # Import GridSearchCV for hyperparameter tuning.
from sklearn.model_selection import cross_val_score # Import cross_val_score for cross-validation.
from sklearn.preprocessing import StandardScaler # Import StandardScaler for feature scaling.
from sklearn.model_selection import train_test_split # Import train_test_split for splitting data into training and testing sets.

# *Combining MLForecast along with Models like LightGBM Regressor & XGBoost Regressor*


*   We combine MLForecast with models like XGBoost or LightGBM because it has in-built feature engineering. It performs feature engineering by automatically creating Lag and Rolling Feature columns for both the historical data and for the future data that the combined model will forecast.




In [37]:
# Display the `df2_continuous` DataFrame.
# This DataFrame contains crime counts for all types, years, and months, with missing values filled, and is ready for feature engineering and model training.
df2_continuous

,TYPE,YEAR,MONTH,Monthly_Crime_Count_Type_wise
0,Break and Enter Commercial,1999,1,303
1,Break and Enter Commercial,1999,2,254
2,Break and Enter Commercial,1999,3,292
3,Break and Enter Commercial,1999,4,266
4,Break and Enter Commercial,1999,5,291
...,...,...,...,...
1399,Vehicle Collision or Pedestrian Struck (with I...,2011,8,148
1400,Vehicle Collision or Pedestrian Struck (with I...,2011,9,142
1401,Vehicle Collision or Pedestrian Struck (with I...,2011,10,134
1402,Vehicle Collision or Pedestrian Struck (with I...,2011,11,158


In [38]:
def add_external_features(df):
    """Add seasonal and temporal features to the DataFrame."""
    df = df.copy()

    # Seasonal features based on month
    df['is_summer'] = df['MONTH'].isin([5, 6, 7, 8]).astype(int) # Indicator for summer months
    df['is_holiday_season'] = df['MONTH'].isin([9, 10, 11, 12]).astype(int) # Indicator for autumn/winter holiday season
    df['is_spring'] = df['MONTH'].isin([1, 2, 3, 4]).astype(int) # Indicator for spring months

    # Quarter feature: divides the year into four 3-month periods
    df['quarter'] = ((df['MONTH'] - 1) // 3) + 1

    # Trigonometric features for capturing smooth seasonality patterns
    df['month_sin'] = np.sin(2 * np.pi * (df['MONTH'] - 1) / 12)
    df['month_cos'] = np.cos(2 * np.pi * (df['MONTH'] - 1) / 12)

    # Month squared (captures non-linear trends related to month)
    df['month_sq'] = df['MONTH'] ** 2

    # Interaction features: combines seasonal indicators with month number
    # This helps capture how the 'strength' or impact of a season changes over its course.
    df['summer_peak'] = (df['is_summer'] * df['MONTH'])
    # The following lines were commented out in the original notebook, they represent alternative interaction features:
    #df['summer_peak'] = (df['is_summer'] * df['Monthly_Crime_Count_Type_wise'])
    #df['holiday_peak'] = (df['is_spring'] * df['Monthly_Crime_Count_Type_wise'])
    df['holiday_peak'] = (df['is_holiday_season'] * df['MONTH'])

    return df

# Apply the feature engineering function to the continuous DataFrame
df2_final_features = add_external_features(df2_continuous)
# Display the DataFrame with new features
df2_final_features

,TYPE,YEAR,MONTH,Monthly_Crime_Count_Type_wise,is_summer,is_holiday_season,is_spring,quarter,month_sin,month_cos,month_sq,summer_peak,holiday_peak
0,Break and Enter Commercial,1999,1,303,0,0,1,1,0.000000,1.000000e+00,1,0,0
1,Break and Enter Commercial,1999,2,254,0,0,1,1,0.500000,8.660254e-01,4,0,0
2,Break and Enter Commercial,1999,3,292,0,0,1,1,0.866025,5.000000e-01,9,0,0
3,Break and Enter Commercial,1999,4,266,0,0,1,2,1.000000,6.123234e-17,16,0,0
4,Break and Enter Commercial,1999,5,291,1,0,0,2,0.866025,-5.000000e-01,25,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,Vehicle Collision or Pedestrian Struck (with I...,2011,8,148,1,0,0,3,-0.500000,-8.660254e-01,64,8,0
1400,Vehicle Collision or Pedestrian Struck (with I...,2011,9,142,0,1,0,3,-0.866025,-5.000000e-01,81,0,9
1401,Vehicle Collision or Pedestrian Struck (with I...,2011,10,134,0,1,0,4,-1.000000,-1.836970e-16,100,0,10
1402,Vehicle Collision or Pedestrian Struck (with I...,2011,11,158,0,1,0,4,-0.866025,5.000000e-01,121,0,11


In [ ]:
# Create a copy of the `df2_final_features` DataFrame and assign it to `Train_df_XGBoost_LightGBM_Models`.
Train_df_XGBoost_LightGBM_Models = df2_final_features.copy()
# Save the DataFrame to a CSV file named 'Train_df_XGBoost_LightGBM_Models.csv' without the index.
Train_df_XGBoost_LightGBM_Models.to_csv('Train_df_XGBoost_LightGBM_Models.csv', index=False)
# Import the `files` module from `google.colab`.
from google.colab import files
# Initiate the download of the CSV file.
files.download('Train_df_XGBoost_LightGBM_Models.csv')

In [39]:
# Filter the `df2_final_features` DataFrame to include data only up to the year 2009.
# This subset (`df2_final2`) is used for initial model training and cross-validation to simulate historical performance.
df2_final2 = df2_final_features[df2_final_features['YEAR'] <= 2009]
# Display the resulting DataFrame.
df2_final2

,TYPE,YEAR,MONTH,Monthly_Crime_Count_Type_wise,is_summer,is_holiday_season,is_spring,quarter,month_sin,month_cos,month_sq,summer_peak,holiday_peak
0,Break and Enter Commercial,1999,1,303,0,0,1,1,0.000000,1.000000e+00,1,0,0
1,Break and Enter Commercial,1999,2,254,0,0,1,1,0.500000,8.660254e-01,4,0,0
2,Break and Enter Commercial,1999,3,292,0,0,1,1,0.866025,5.000000e-01,9,0,0
3,Break and Enter Commercial,1999,4,266,0,0,1,2,1.000000,6.123234e-17,16,0,0
4,Break and Enter Commercial,1999,5,291,1,0,0,2,0.866025,-5.000000e-01,25,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,Vehicle Collision or Pedestrian Struck (with I...,2009,8,131,1,0,0,3,-0.500000,-8.660254e-01,64,8,0
1376,Vehicle Collision or Pedestrian Struck (with I...,2009,9,128,0,1,0,3,-0.866025,-5.000000e-01,81,0,9
1377,Vehicle Collision or Pedestrian Struck (with I...,2009,10,154,0,1,0,4,-1.000000,-1.836970e-16,100,0,10
1378,Vehicle Collision or Pedestrian Struck (with I...,2009,11,110,0,1,0,4,-0.866025,5.000000e-01,121,0,11


In [40]:
# Filter the `df2_final_features` DataFrame to include data only from the year 2010 onwards.
# This subset (`df2_final1`) will be used as the actual 'future' period against which forecasts will be compared or extended.
df2_final1 = df2_final_features[df2_final_features['YEAR'] >= 2010]
# Display the resulting DataFrame.
df2_final1

,TYPE,YEAR,MONTH,Monthly_Crime_Count_Type_wise,is_summer,is_holiday_season,is_spring,quarter,month_sin,month_cos,month_sq,summer_peak,holiday_peak
132,Break and Enter Commercial,2010,1,185,0,0,1,1,0.000000,1.000000e+00,1,0,0
133,Break and Enter Commercial,2010,2,163,0,0,1,1,0.500000,8.660254e-01,4,0,0
134,Break and Enter Commercial,2010,3,184,0,0,1,1,0.866025,5.000000e-01,9,0,0
135,Break and Enter Commercial,2010,4,199,0,0,1,2,1.000000,6.123234e-17,16,0,0
136,Break and Enter Commercial,2010,5,195,1,0,0,2,0.866025,-5.000000e-01,25,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,Vehicle Collision or Pedestrian Struck (with I...,2011,8,148,1,0,0,3,-0.500000,-8.660254e-01,64,8,0
1400,Vehicle Collision or Pedestrian Struck (with I...,2011,9,142,0,1,0,3,-0.866025,-5.000000e-01,81,0,9
1401,Vehicle Collision or Pedestrian Struck (with I...,2011,10,134,0,1,0,4,-1.000000,-1.836970e-16,100,0,10
1402,Vehicle Collision or Pedestrian Struck (with I...,2011,11,158,0,1,0,4,-0.866025,5.000000e-01,121,0,11


In [41]:
# Install the `mlforecast` library.
# This library is crucial for time series forecasting, especially when combined with machine learning models like LightGBM and XGBoost.
!pip install mlforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 19.2 MB/s eta 0:00:00


# **LightGBM Regressor Model + MLForecast**

# *`Model Description:`*
LightGBM (Light Gradient Boosting Machine)

LightGBM is a gradient boosting framework developed by Microsoft that builds decision trees using a leaf-wise tree growth strategy. Unlike traditional level-wise tree growth, LightGBM splits the leaf with the highest loss reduction, allowing it to capture complex patterns more efficiently.

# *`Key Characteristics:`*

* Gradient boosting-based ensemble learning method

* Leaf-wise tree growth strategy

* High training speed and low memory consumption

* Efficient handling of large datasets

* Strong performance with feature-engineered data

* Excellent at capturing nonlinear relationships

# *`Why LightGBM Was Used in This Project:`*

In this project, LightGBM was used to forecast monthly crime counts for each crime type. Since the dataset included lag features, rolling statistics, and seasonal indicators, LightGBM was particularly suitable due to:

* Its ability to handle complex time-series features

* Strong modeling of nonlinear temporal relationships

* Computational efficiency for multi-series forecasting

# *Model Training & Forecast*

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from mlforecast import MLForecast
from mlforecast.lag_transforms import RollingMean, RollingStd
from mlforecast.target_transforms import AutoDifferences
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# 1. DATA PREPARATION
# Select a subset of the data (`df2_final2`, which contains data up to 2009) for initial training and cross-validation.
df = df2_final2.copy()

# Create a datetime column 'ds' from 'YEAR' and 'MONTH' for time series functionality.
df['ds'] = pd.to_datetime(df['YEAR'].astype(str) + '-' + df['MONTH'].astype(str) + '-01')

# Rename columns to 'unique_id' (for crime type) and 'y' (for crime count) as required by MLForecast.
df = df.rename(columns={'TYPE': 'unique_id', 'Monthly_Crime_Count_Type_wise': 'y'})

# Select the final training features, including 'unique_id', 'ds', 'y', and other engineered temporal features.
df_train = df[['unique_id', 'ds', 'y', 'MONTH', 'is_summer', 'is_holiday_season', 'is_spring', 'quarter', 'month_sin', 'month_cos', 'month_sq', 'summer_peak', 'holiday_peak']].copy()

# 2. DEFINE THE MODEL & FEATURES
# Initialize MLForecast with an LGBMRegressor model.
# 'freq' is set to 'MS' (Monthly Start) to specify the time series frequency.
# 'lags' are defined to capture past values as features.
# 'lag_transforms' apply rolling statistics (mean, std) over various windows to the lag features.
# 'date_features' tells the model to extract features from the date column (e.g., month of year).
# 'target_transforms' apply differencing to handle seasonality and trends in the target variable.
fcst = MLForecast(
    models=[lgb.LGBMRegressor(n_estimators=1200, learning_rate=0.02, random_state=42, verbosity=-1)],
    freq='MS',
    lags=[1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 18, 24],
    lag_transforms={
        1: [RollingMean(window_size=3), RollingStd(window_size=3), RollingMean(window_size=6), RollingStd(window_size=6), RollingMean(window_size=12), RollingStd(window_size=12), RollingMean(window_size=24)]
    },
    date_features=['month'],
    target_transforms=[AutoDifferences(max_diffs=2)]
)

# 3. PERFORM CROSS-VALIDATION (The "Evaluation" Step)
# Perform cross-validation to evaluate the model's performance on historical data.
# 'h=24' specifies a 24-month forecast horizon for each window.
# 'n_windows=3' creates 3 separate validation windows.
# 'step_size=24' moves the training window forward by 24 months for each subsequent window.
# 'static_features=[]' indicates no static features are used at the 'unique_id' level.
cv_results = fcst.cross_validation(
    df=df_train,
    h=24,
    n_windows=3,
    step_size=24,
    static_features=[]
)

# Define a function to calculate Mean Absolute Error (MAE) and Mean Absolute Percentage Error (MAPE).
def calculate_metrics(df_cv, model_name):
    actual = df_cv['y']
    pred = df_cv[model_name]
    mae = mean_absolute_error(actual, pred)
    mape = mean_absolute_percentage_error(actual, pred)*100
    return mae, mape

# Calculate and print global MAE and MAPE from cross-validation results.
mae, mape = calculate_metrics(cv_results, 'LGBMRegressor')
print(f"Cross-Validation Results -> MAE: {mae:.3f}, MAPE: {mape:.2f}%")

# Define a function to calculate Weighted Mean Absolute Percentage Error (WMAPE).
def get_wmape(df_cv, model_col='LGBMRegressor'):
    total_abs_error = np.sum(np.abs(df_cv['y'] - df_cv[model_col]))
    total_actuals = np.sum(df_cv['y'])
    wmape = (total_abs_error / total_actuals) * 100
    return wmape

# Calculate and print the global WMAPE.
score = get_wmape(cv_results)
print(f"Global wMAPE: {score:.2f}%")

# 4. Calculating MAE, MAPE per crime type
# Define a function to calculate MAE and MAPE for each unique crime type.
def per_crime_type_metrics(df, model_col='LGBMRegressor'):
    results = []
    grouped = df.groupby('unique_id')
    for uid, group in grouped:
        actual = group['y']
        pred = group[model_col]
        mae = mean_absolute_error(actual, pred)
        mape = mean_absolute_percentage_error(actual, pred) * 100
        results.append({'unique_id': uid, 'MAE': mae, 'MAPE_%': mape})
    result_df = pd.DataFrame(results)
    return result_df.sort_values('MAPE_%', ascending=False).reset_index(drop=True)

# Calculate and print metrics per crime type.
metrics_per_type = per_crime_type_metrics(cv_results, model_col='LGBMRegressor')
print(metrics_per_type)

# Define a function to calculate WMAPE per unique crime type.
def per_id_wmape(df_cv, model_col='LGBMRegressor'):
    summary = df_cv.groupby('unique_id').apply(
        lambda x: (np.sum(np.abs(x['y'] - x[model_col])) / np.sum(x['y'])) * 100
    ).reset_index()
    summary.columns = ['unique_id', 'wMAPE']
    return summary

# Calculate and print WMAPE per crime type.
type_analysis = per_id_wmape(cv_results)
print(type_analysis.sort_values('wMAPE'))

# 5. FINAL FIT & FUTURE PREDICTION
# Fit the MLForecast model on the entire training dataset (`df_train`) before forecasting.
# 'max_horizon=24' specifies that the model should be prepared to forecast up to 24 steps ahead.
fcst.fit(df_train, static_features=[], max_horizon=24)

# Create a DataFrame `X_df` for future dynamic features for the next 24 months.
# This involves generating future dates and then re-creating the seasonal and temporal features.
last_date = df_train['ds'].max()
future_dates = pd.date_range(start=last_date + pd.offsets.MonthBegin(1), periods=24, freq='MS')
uids = df_train['unique_id'].unique()

X_df = pd.DataFrame({
    'unique_id': [i for i in uids for _ in range(24)],
    'ds': list(future_dates) * len(uids)
})
X_df['MONTH'] = X_df['ds'].dt.month
X_df['is_summer'] = X_df['MONTH'].isin([5, 6, 7, 8]).astype(int)
X_df['is_holiday_season'] = X_df['MONTH'].isin([9, 10, 11, 12]).astype(int)
X_df['is_spring'] = X_df['MONTH'].isin([1, 2, 3, 4]).astype(int)
X_df['quarter'] = ((X_df['MONTH'] - 1) // 3) + 1
X_df['month_sin'] = np.sin(2 * np.pi * (X_df['MONTH'] - 1)/ 12)
X_df['month_cos'] = np.cos(2 * np.pi * (X_df['MONTH'] - 1)/ 12)
X_df['month_sq'] = X_df['MONTH'] ** 2
X_df['summer_peak'] = X_df['is_summer'] * X_df['MONTH']
X_df['holiday_peak'] = X_df['is_holiday_season'] * X_df['MONTH']

# Generate the final 24-month forecast using the fitted model and future features.
final_forecast2 = fcst.predict(h=24, X_df=X_df)
print("\nFinal 24-Month Forecast:")
print(final_forecast2.head())

Cross-Validation Results -> MAE: 36.809, MAPE: 16.50%
Global wMAPE: 12.97%
                                           unique_id        MAE     MAPE_%
0                                   Theft of Bicycle  32.800533  28.422463
1                                   Theft of Vehicle  34.369652  28.222084
2  Vehicle Collision or Pedestrian Struck (with I...  21.179979  20.126548
3                         Break and Enter Commercial  30.740988  19.636919
4                  Break and Enter Residential/Other  36.821529  13.206689
5                                           Mischief  46.389175  11.665858
6                                 Theft from Vehicle  80.238853  11.288932
7                           Offence Against a Person  25.747179   8.003601
8                                        Other Theft  22.994766   7.963970
                                           unique_id      wMAPE
4                                        Other Theft   7.915582
3                           Offence Against a P

In [42]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from mlforecast import MLForecast
from mlforecast.lag_transforms import RollingMean, RollingStd
from mlforecast.target_transforms import AutoDifferences, LocalStandardScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# 1. DATA PREPARATION
# Use the `df2_final_features` DataFrame, which contains all years and engineered features.
df = df2_final_features.copy()

# Create a datetime column 'ds' from 'YEAR' and 'MONTH' for time series functionality.
df['ds'] = pd.to_datetime(df['YEAR'].astype(str) + '-' + df['MONTH'].astype(str) + '-01')
# Rename columns to 'unique_id' (for crime type) and 'y' (for crime count) as required by MLForecast.
df = df.rename(columns={'TYPE': 'unique_id', 'Monthly_Crime_Count_Type_wise': 'y'})

# Select the final training features, including 'unique_id', 'ds', 'y', and other engineered temporal features.
df_train = df[['unique_id', 'ds', 'y', 'MONTH', 'is_summer', 'is_holiday_season', 'is_spring', 'quarter', 'month_sin', 'month_cos', 'month_sq', 'summer_peak', 'holiday_peak']].copy()

# 2. DEFINE THE MODEL & FEATURES
# Initialize MLForecast with an LGBMRegressor model.
# 'freq' is set to 'MS' (Monthly Start) to specify the time series frequency.
# 'lags' are defined to capture past values as features (e.g., crime counts from previous months).
# 'lag_transforms' apply rolling statistics (mean, std) over various windows to the lag features.
# 'date_features' tells the model to extract features from the date column (e.g., month of year).
# 'target_transforms' apply differencing to handle seasonality and trends in the target variable.
fcst = MLForecast(
    models=[lgb.LGBMRegressor(n_estimators=1200, learning_rate=0.02, random_state=42, verbosity=-1)],
    freq='MS',                # Monthly Start frequency
    lags=[1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 18, 24],          # Lags selected to capture various seasonal and short-term dependencies
    lag_transforms={
        1: [RollingMean(window_size=3), RollingStd(window_size=3), RollingMean(window_size=6), RollingStd(window_size=6), RollingMean(window_size=12), RollingStd(window_size=12), RollingMean(window_size=24)] # Rolling statistics to capture trends and volatility
    },
    date_features=['month'],  # Use month as a feature directly
    target_transforms=[AutoDifferences(max_diffs=2)] # Apply auto-differencing up to 2 times to stabilize the series
)

# 3. PERFORM CROSS-VALIDATION (The "Evaluation" Step)
# Perform cross-validation to evaluate the model's performance on historical data.
# 'h=24' specifies a 24-month forecast horizon for each validation window.
# 'n_windows=3' creates 3 separate validation windows for robust evaluation.
# 'step_size=24' moves the training window forward by 24 months for each subsequent window.
# 'static_features=[]' indicates no static features are used at the 'unique_id' level (crime type).
cv_results = fcst.cross_validation(
    df=df_train,
    h=24,
    n_windows=3,
    step_size=24,
    static_features=[]
)

# Define a function to calculate Mean Absolute Error (MAE) and Mean Absolute Percentage Error (MAPE).
def calculate_metrics(df_cv, model_name):
    actual = df_cv['y']
    pred = df_cv[model_name]
    mae = mean_absolute_error(actual, pred)
    mape = mean_absolute_percentage_error(actual, pred)*100
    return mae, mape

# Calculate and print global MAE and MAPE from cross-validation results.
mae, mape = calculate_metrics(cv_results, 'LGBMRegressor')
print(f"Cross-Validation Results -> MAE: {mae:.3f}, MAPE: {mape:.2f}%")

import numpy as np

def get_wmape(df_cv, model_col='LGBMRegressor'):
    # Sum of all absolute errors across all months and crime types.
    total_abs_error = np.sum(np.abs(df_cv['y'] - df_cv[model_col]))

    # Sum of all actual crime counts.
    total_actuals = np.sum(df_cv['y'])

    # Calculate weighted percentage.
    wmape = (total_abs_error / total_actuals) * 100
    return wmape

# Calculate and print the global WMAPE.
score = get_wmape(cv_results)
print(f"Global wMAPE: {score:.2f}%")


#4. Calculating MAE, MAPE per crime type

def per_crime_type_metrics(df, model_col='LGBMRegressor'):
    """
    Calculates MAE and MAPE per unique_id (crime type)
    using sklearn metric functions.
    """

    results = []

    grouped = df.groupby('unique_id')

    for uid, group in grouped:
        actual = group['y']
        pred = group[model_col]

        mae = mean_absolute_error(actual, pred)
        mape = mean_absolute_percentage_error(actual, pred) * 100

        results.append({
            'unique_id': uid,
            'MAE': mae,
            'MAPE_%': mape
        })

    result_df = pd.DataFrame(results)
    return result_df.sort_values('MAPE_%', ascending=False).reset_index(drop=True)

#Usage

metrics_per_type = per_crime_type_metrics(cv_results, model_col='LGBMRegressor')
print(metrics_per_type)

#Calculating WMAPE per crime type

def per_id_wmape(df_cv, model_col='LGBMRegressor'):
    # Group by unique_id and calculate (sum of errors / sum of actuals).
    summary = df_cv.groupby('unique_id').apply(
        lambda x: (np.sum(np.abs(x['y'] - x[model_col])) / np.sum(x['y'])) * 100
    ).reset_index()

    summary.columns = ['unique_id', 'wMAPE']
    return summary

# Usage
type_analysis = per_id_wmape(cv_results)
print(type_analysis.sort_values('wMAPE'))


# 5. FINAL FIT & FUTURE PREDICTION
# After verifying performance, train on ALL data and project 2 years ahead.
fcst.fit(df_train, static_features=[], max_horizon=24)

# Create future dynamic features (X_df) for the next 24 months.
# This involves generating future dates and then re-creating the seasonal and temporal features.
last_date = df_train['ds'].max()
future_dates = pd.date_range(start=last_date + pd.offsets.MonthBegin(1), periods=24, freq='MS')
uids = df_train['unique_id'].unique()

X_df = pd.DataFrame({
    'unique_id': [i for i in uids for _ in range(24)],
    'ds': list(future_dates) * len(uids)
})
X_df['MONTH'] = X_df['ds'].dt.month
X_df['is_summer'] = X_df['MONTH'].isin([5, 6, 7, 8]).astype(int)
X_df['is_holiday_season'] = X_df['MONTH'].isin([9, 10, 11, 12]).astype(int)
X_df['is_spring'] = X_df['MONTH'].isin([1, 2, 3, 4]).astype(int)
X_df['quarter'] = ((X_df['MONTH'] - 1) // 3) + 1
X_df['month_sin'] = np.sin(2 * np.pi * (X_df['MONTH'] - 1)/ 12)
X_df['month_cos'] = np.cos(2 * np.pi * (X_df['MONTH'] - 1)/ 12)
X_df['month_sq'] = X_df['MONTH'] ** 2
X_df['summer_peak'] = X_df['is_summer'] * X_df['MONTH']
X_df['holiday_peak'] = X_df['is_holiday_season'] * X_df['MONTH']

# Final Forecast Generation.
final_forecast1 = fcst.predict(h=24, X_df=X_df)
print("\nFinal 24-Month Forecast:")
print(final_forecast1.head())

Cross-Validation Results -> MAE: 45.515, MAPE: 16.87%
Global wMAPE: 16.08%
                                           unique_id         MAE     MAPE_%
0                                   Theft of Vehicle   26.538380  25.801134
1                                   Theft of Bicycle   46.222271  25.334266
2                         Break and Enter Commercial   33.452271  19.098346
3  Vehicle Collision or Pedestrian Struck (with I...   22.936720  18.868941
4                                 Theft from Vehicle  139.179459  17.497714
5                  Break and Enter Residential/Other   34.007759  12.481236
6                           Offence Against a Person   34.233402  12.460000
7                                        Other Theft   36.165554  10.726412
8                                           Mischief   36.900856   9.521243
                                           unique_id      wMAPE
2                                           Mischief  10.033843
4                                    

In [43]:
final_forecast6 = final_forecast1.copy()
# Rename columns to a more descriptive format for clarity and consistency.
final_forecast6.rename(columns={'unique_id':'TYPE', 'ds':'Date', 'LGBMRegressor': 'Monthly_Crime_Count_Type_wise'}, inplace=True)
# Extract the year from the 'Date' column and create a new 'YEAR' column.
final_forecast6['YEAR'] = final_forecast6['Date'].dt.year
# Extract the month from the 'Date' column and create a new 'MONTH' column.
final_forecast6['MONTH'] = final_forecast6['Date'].dt.month
# Select and reorder the relevant columns for the final output.
final_forecast6 = final_forecast6[['TYPE', 'YEAR', 'MONTH', 'Monthly_Crime_Count_Type_wise']]
# Sort the DataFrame by crime type, year, and month to ensure chronological order for each type.
final_forecast6.sort_values(by=['TYPE', 'YEAR', 'MONTH'], inplace=True)
# Reset the index of the DataFrame after sorting.
final_forecast6.reset_index(drop=True, inplace=True)
# Display the processed forecast DataFrame.
final_forecast6

,TYPE,YEAR,MONTH,Monthly_Crime_Count_Type_wise
0,Break and Enter Commercial,2012,1,223.275696
1,Break and Enter Commercial,2012,2,184.468704
2,Break and Enter Commercial,2012,3,198.991211
3,Break and Enter Commercial,2012,4,178.404907
4,Break and Enter Commercial,2012,5,192.416641
...,...,...,...,...
211,Vehicle Collision or Pedestrian Struck (with I...,2013,8,155.484039
212,Vehicle Collision or Pedestrian Struck (with I...,2013,9,167.898621
213,Vehicle Collision or Pedestrian Struck (with I...,2013,10,155.274078
214,Vehicle Collision or Pedestrian Struck (with I...,2013,11,130.742920


# *Saving the LightGBM Model Forecasted Data for Streamlit Visualisations*

In [ ]:
# Define a function to convert numerical month values (1-12) to their abbreviated names.
def get_month_name(month_num):
  if month_num == 1:
    return 'Jan'
  elif month_num == 2:
    return 'Feb'
  elif month_num == 3:
    return 'Mar'
  elif month_num == 4:
    return 'Apr'
  elif month_num == 5:
    return 'May'
  elif month_num == 6:
    return 'Jun'
  elif month_num == 7:
    return 'Jul'
  elif month_num == 8:
    return 'Aug'
  elif month_num == 9:
    return 'Sep'
  elif month_num == 10:
    return 'Oct'
  elif month_num == 11:
    return 'Nov'
  else:
    return 'Dec'

# Create a copy of `final_forecast1` for further processing and visualization.
final_forecast7 = final_forecast1.copy()
# Rename columns for clarity, changing 'unique_id' to 'TYPE', 'ds' to 'Date', and the model's prediction column to 'Crime_Count'.
final_forecast7.rename(columns={'unique_id':'TYPE', 'ds':'Date', 'LGBMRegressor': 'Crime_Count'}, inplace=True)
# Extract the year from the 'Date' column and create a new 'YEAR' column.
final_forecast7['YEAR'] = final_forecast7['Date'].dt.year
# Extract the month from the 'Date' column and create a new 'MONTH' column.
final_forecast7['MONTH'] = final_forecast7['Date'].dt.month
# Initialize 'Month_name' column with NaN values.
final_forecast7['Month_name'] = np.nan
# Apply the `get_month_name` function to populate the 'Month_name' column based on 'MONTH'.
final_forecast7['Month_name'] = final_forecast7['MONTH'].apply(get_month_name)
# Select and reorder the relevant columns for the final output.
final_forecast7 = final_forecast7[['TYPE', 'YEAR', 'MONTH', 'Month_name', 'Crime_Count']]
# Sort the DataFrame by crime type, year, month, and month name for consistent ordering.
final_forecast7.sort_values(by=['TYPE', 'YEAR', 'MONTH', 'Month_name'], inplace=True)
# Reset the index of the DataFrame after sorting.
final_forecast7.reset_index(drop=True, inplace=True)

# Group the original `Train_df` to get past crime counts by type, year, month, and month name.
Past_df_type_wise_crime_count = Train_df.groupby(['TYPE', 'YEAR', 'MONTH', 'Month_name']).size().reset_index(name='Crime_Count')
# Apply the `get_month_name` function to ensure consistent month names in the past data.
Past_df_type_wise_crime_count['Month_name'] = Past_df_type_wise_crime_count['MONTH'].apply(get_month_name)
# Sort the past crime count DataFrame for consistent ordering.
Past_df_type_wise_crime_count.sort_values(by=['TYPE', 'YEAR', 'MONTH', 'Month_name'], ascending=[True, True, True, True], inplace=True)
# Reset the index.
Past_df_type_wise_crime_count.reset_index(drop=True, inplace=True)

# Concatenate the past crime data with the LightGBM forecasted data.
# This creates a single DataFrame containing both historical and predicted crime counts.
Past_and_Future_df_type_wise_crime_count_LightBGM = pd.concat([Past_df_type_wise_crime_count, final_forecast7], ignore_index=True, axis=0)
# Display the combined DataFrame.
Past_and_Future_df_type_wise_crime_count_LightBGM

,TYPE,YEAR,MONTH,Month_name,Crime_Count
0,Break and Enter Commercial,1999,1,Jan,303.000000
1,Break and Enter Commercial,1999,2,Feb,254.000000
2,Break and Enter Commercial,1999,3,Mar,292.000000
3,Break and Enter Commercial,1999,4,Apr,266.000000
4,Break and Enter Commercial,1999,5,May,291.000000
...,...,...,...,...,...
1615,Vehicle Collision or Pedestrian Struck (with I...,2013,8,Aug,155.484039
1616,Vehicle Collision or Pedestrian Struck (with I...,2013,9,Sep,167.898621
1617,Vehicle Collision or Pedestrian Struck (with I...,2013,10,Oct,155.274078
1618,Vehicle Collision or Pedestrian Struck (with I...,2013,11,Nov,130.742920


In [ ]:
from google.colab import files
# Save the combined historical and LightGBM forecasted DataFrame to a CSV file.
Past_and_Future_df_type_wise_crime_count_LightBGM.to_csv('Past_and_Future_df_type_wise_crime_count_LightBGM.csv', index=False)
# Initiate the download of the CSV file to the local machine.
files.download('Past_and_Future_df_type_wise_crime_count_LightBGM.csv')

# *Visualisations on the LightGBM Regressor Model Forecasted Data*

In [44]:
# Create a copy of `final_forecast1` to prepare for yearly visualization.
final_forecast6 = final_forecast1.copy()
# Rename columns for clarity and consistency.
final_forecast6.rename(columns={'unique_id':'TYPE', 'ds':'Date', 'LGBMRegressor': 'Monthly_Crime_Count_Type_wise'}, inplace=True)
# Extract year and month from the 'Date' column.
final_forecast6['YEAR'] = final_forecast6['Date'].dt.year
final_forecast6['MONTH'] = final_forecast6['Date'].dt.month
# Select and reorder relevant columns.
final_forecast6 = final_forecast6[['TYPE', 'YEAR', 'MONTH', 'Monthly_Crime_Count_Type_wise']]
# Sort the DataFrame.
final_forecast6.sort_values(by=['TYPE', 'YEAR', 'MONTH'], inplace=True)
# Reset the index.
final_forecast6.reset_index(drop=True, inplace=True)

# Group the forecasted data by 'TYPE' and 'YEAR' to get total yearly crime counts for the future.
Yearly_type_wise_crime_count_future = final_forecast6.groupby(['TYPE', 'YEAR'])['Monthly_Crime_Count_Type_wise'].sum().reset_index(name='Crime Count')
# Sort the future crime count DataFrame.
Yearly_type_wise_crime_count_future.sort_values(by=['TYPE', 'YEAR'], ascending=[True, True], inplace=True)
# Reset the index.
Yearly_type_wise_crime_count_future.reset_index(drop=True, inplace=True)

# Group the original `Train_df` by 'TYPE' and 'YEAR' to get total yearly crime counts for the past.
Yearly_type_wise_crime_count = Train_df.groupby(['TYPE', 'YEAR']).size().reset_index(name='Crime Count')
# Sort the past crime count DataFrame.
Yearly_type_wise_crime_count.sort_values(by=['TYPE', 'YEAR'], ascending=[True, True], inplace=True)
# Reset the index.
Yearly_type_wise_crime_count.reset_index(drop=True, inplace=True)

# Concatenate past and future yearly crime counts.
df = pd.concat([Yearly_type_wise_crime_count, Yearly_type_wise_crime_count_future], ignore_index=True, axis=0)

# Create a line plot to visualize type-wise crime counts over past and future years.
fig = px.line(df, x='YEAR', y='Crime Count', color='TYPE', title='Type Wise Crime Count Over the Past Years & the Future Years', labels={'YEAR':'Year'})
# Update plot layout for aesthetics.
fig.update_layout(xaxis_title='Year', yaxis_title='Crime Count', title_x = 0.5, title_font_size=24, title_font_family='Arial Black', width=1200, height=700)
# Display the plot.
fig.show()

***`Trend Observed:`***

Forecasted crime trends closely follow historical trends.

***`Key Insight:`***

LightGBM successfully captured temporal and seasonal dependencies.

In [45]:
# Create a copy of `final_forecast1` for processing.
final_forecast6 = final_forecast1.copy()
# Rename columns for clarity.
final_forecast6.rename(columns={'unique_id':'TYPE', 'ds':'Date', 'LGBMRegressor': 'Monthly_Crime_Count_Type_wise'}, inplace=True)
# Extract year and month.
final_forecast6['YEAR'] = final_forecast6['Date'].dt.year
final_forecast6['MONTH'] = final_forecast6['Date'].dt.month
# Select and reorder columns.
final_forecast6 = final_forecast6[['TYPE', 'YEAR', 'MONTH', 'Monthly_Crime_Count_Type_wise']]
# Sort the DataFrame.
final_forecast6.sort_values(by=['TYPE', 'YEAR', 'MONTH'], inplace=True)
# Reset the index.
final_forecast6.reset_index(drop=True, inplace=True)

# Create a copy for future monthly data and initialize 'Month_name'.
df1 = final_forecast6.copy()
df1['Month_name'] = np.nan

# Define a function to get abbreviated month names.
def get_month_name(month_num):
  if month_num == 1:
    return 'Jan'
  elif month_num == 2:
    return 'Feb'
  elif month_num == 3:
    return 'Mar'
  elif month_num == 4:
    return 'Apr'
  elif month_num == 5:
    return 'May'
  elif month_num == 6:
    return 'Jun'
  elif month_num == 7:
    return 'Jul'
  elif month_num == 8:
    return 'Aug'
  elif month_num == 9:
    return 'Sep'
  elif month_num == 10:
    return 'Oct'
  elif month_num == 11:
    return 'Nov'
  else:
    return 'Dec'

# Apply the function to populate 'Month_name' in the future data.
df1['Month_name'] = df1['MONTH'].apply(get_month_name)
# Rename 'Monthly_Crime_Count_Type_wise' to 'Crime_Count'.
df1.rename(columns={'Monthly_Crime_Count_Type_wise':'Crime_Count'}, inplace=True)
# Sort the future monthly data.
df1.sort_values(by=['TYPE', 'YEAR', 'MONTH', 'Month_name'], inplace=True)
# Reset the index.
df1.reset_index(drop=True, inplace=True)


# Group the original `Train_df` to get past monthly crime counts by type, year, and month name.
df2 = Train_df.groupby(['TYPE', 'YEAR', 'MONTH', 'Month_name']).size().reset_index(name='Crime_Count')
# Apply the `get_month_name` function to ensure consistent month names in past data.
df2['Month_name'] = df2['MONTH'].apply(get_month_name)
# Rename columns for consistency.
df2.columns=['TYPE', 'YEAR', 'MONTH', 'Month_name', 'Crime_Count']
# Sort the past monthly data.
df2.sort_values(by=['TYPE', 'YEAR', 'MONTH', 'Month_name'], ascending=[True, True, True, True], inplace=True)
# Reset the index.
df2.reset_index(drop=True, inplace=True)

# Concatenate past and future monthly crime counts.
df = pd.concat([df2, df1], ignore_index=True, axis=0)

# Create a faceted line plot to visualize monthly crime counts for past and future years.
# Each facet represents a crime type, and lines are colored by year.
fig = px.line(df, x='Month_name', y='Crime_Count', color='YEAR', title='Crime Count Peak Months of Past & Future Years(Predicted Using LightBGM Regressor)', facet_col='TYPE', facet_col_wrap=3,
             labels={'MONTH': 'Month', 'Crime_Count': 'Crime Count'})
# Update plot layout for aesthetics.
fig.update_layout(xaxis_title='Month', yaxis_title='Crime Count', title_x=0.5, title_font_size=24, title_font_family='Arial Black', width=1200, height=1200)
# Display the plot.
fig.show()

***`Trend Observed:`***

Forecast preserves historical monthly seasonal pattern.

***`Key Insight:`***

LightGBM provides stable and realistic monthly forecasts.

# **XGBoost Regressor Model + MLForecast**

# *`Model Description:`*

***XGBoost (Extreme Gradient Boosting)***

XGBoost is an optimized gradient boosting algorithm designed for speed and performance. It uses a level-wise tree growth strategy, making it more conservative and often less prone to overfitting compared to leaf-wise approaches.

# *`Key Characteristics:`*

* Gradient boosting ensemble model

* Level-wise tree growth

* Strong regularization (L1 and L2)

* Robust to noise

* Highly scalable and widely used in machine learning competitions

# *`Why XGBoost Was Used in This Project:`*

XGBoost was applied to forecast monthly crime counts per crime type, similar to LightGBM. It was selected because:

* It effectively captures nonlinear relationships

* It provides robust performance with engineered time-series features

* It is well-established for predictive modeling tasks

# *Model Training & Forecast*

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from mlforecast import MLForecast
from mlforecast.lag_transforms import RollingMean, RollingStd
from mlforecast.target_transforms import AutoDifferences, LocalStandardScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# 1. DATA PREPARATION
# Use the `df2_final2` DataFrame, which contains data up to 2009 for initial training.
df = df2_final2.copy()

# Create a datetime column 'ds' from 'YEAR' and 'MONTH' for time series functionality.
df['ds'] = pd.to_datetime(df['YEAR'].astype(str) + '-' + df['MONTH'].astype(str) + '-01')
# Rename columns to 'unique_id' (for crime type) and 'y' (for crime count) as required by MLForecast.
df = df.rename(columns={'TYPE': 'unique_id', 'Monthly_Crime_Count_Type_wise': 'y'})

# Select the final training features, including 'unique_id', 'ds', 'y', and other engineered temporal features.
df_train = df[['unique_id', 'ds', 'y', 'MONTH', 'is_summer', 'is_holiday_season', 'is_spring', 'quarter', 'month_sin', 'month_cos', 'month_sq', 'summer_peak', 'holiday_peak']].copy()

# 2. DEFINE THE MODEL & FEATURES
# Initialize MLForecast with an XGBRegressor model.
# 'freq' is set to 'MS' (Monthly Start) to specify the time series frequency.
# 'lags' are defined to capture past values as features.
# 'lag_transforms' apply rolling statistics (mean, std) over various windows to the lag features.
# 'date_features' tells the model to extract features from the date column (e.g., month of year).
# 'target_transforms' apply differencing to handle seasonality and trends in the target variable.
fcst = MLForecast(
    models=[xgb.XGBRegressor(n_estimators=1200, learning_rate=0.01, random_state=42, verbosity=0)],
    freq='MS',                # Monthly Start frequency
    lags=[1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 18, 24],          # Lags chosen to capture various temporal dependencies
    lag_transforms={
        1: [RollingMean(window_size=3), RollingStd(window_size=3), RollingMean(window_size=6), RollingStd(window_size=6), RollingMean(window_size=12), RollingStd(window_size=12), RollingMean(window_size=24)] # Rolling statistics for trend and volatility
    },
    date_features=['month'],  # Use month as a feature
    target_transforms=[AutoDifferences(max_diffs=2)] # Apply auto-differencing
)

# 3. PERFORM CROSS-VALIDATION (The "Evaluation" Step)
# This simulates 3 separate 24-month forecasts from the historical data up to 2009.
cv_results = fcst.cross_validation(
    df=df_train,
    h=24,
    n_windows=3,
    step_size=24,
    static_features=[] # Treats 'Month' as a dynamic variable
)

# Calculate Evaluation Metrics
# Define a function to calculate Mean Absolute Error (MAE) and Mean Absolute Percentage Error (MAPE).
def calculate_metrics(df_cv, model_name):
    actual = df_cv['y']
    pred = df_cv[model_name]
    mae = mean_absolute_error(actual, pred)
    mape = mean_absolute_percentage_error(actual, pred)*100
    return mae, mape

# Calculate and print global MAE and MAPE from cross-validation results.
mae, mape = calculate_metrics(cv_results, 'XGBRegressor')
print(f"Cross-Validation Results -> MAE: {mae:.3f}, MAPE: {mape:.2f}%")

import numpy as np

# Define a function to calculate global Weighted Mean Absolute Percentage Error (WMAPE).
def get_wmape(df_cv, model_col='XGBRegressor'):
    # Sum of all absolute errors across all months and crime types.
    total_abs_error = np.sum(np.abs(df_cv['y'] - df_cv[model_col]))

    # Sum of all actual crime counts.
    total_actuals = np.sum(df_cv['y'])

    # Calculate weighted percentage.
    wmape = (total_abs_error / total_actuals) * 100
    return wmape

# Usage
score = get_wmape(cv_results)
print(f"Global wMAPE: {score:.2f}%")


#4. Calculating MAE, MAPE per crime type

def per_crime_type_metrics(df, model_col='XGBRegressor'):
    """
    Calculates MAE and MAPE per unique_id (crime type)
    using sklearn metric functions.
    """

    results = []

    grouped = df.groupby('unique_id')

    for uid, group in grouped:
        actual = group['y']
        pred = group[model_col]
        mae = mean_absolute_error(actual, pred)
        mape = mean_absolute_percentage_error(actual, pred) * 100

        results.append({
            'unique_id': uid,
            'MAE': mae,
            'MAPE_%': mape
        })

    result_df = pd.DataFrame(results)
    return result_df.sort_values('MAPE_%', ascending=False).reset_index(drop=True)

#Usage

metrics_per_type = per_crime_type_metrics(cv_results, model_col='XGBRegressor')
print(metrics_per_type)

#Calculating WMAPE per crime type

def per_id_wmape(df_cv, model_col='XGBRegressor'):
    # Group by unique_id and calculate (sum of errors / sum of actuals).
    summary = df_cv.groupby('unique_id').apply(
        lambda x: (np.sum(np.abs(x['y'] - x[model_col])) / np.sum(x['y'])) * 100
    ).reset_index()

    summary.columns = ['unique_id', 'wMAPE']
    return summary

# Usage
type_analysis = per_id_wmape(cv_results)
print(type_analysis.sort_values('wMAPE'))


# 5. FINAL FIT & FUTURE PREDICTION
# After verifying performance, train on ALL data (up to 2009 in df_train) and project 2 years ahead.
fcst.fit(df_train, static_features=[], max_horizon=24)

# Create future dynamic features (X_df) for the next 24 months (2010-2011) based on the last date in df_train.
last_date = df_train['ds'].max()
future_dates = pd.date_range(start=last_date + pd.offsets.MonthBegin(1), periods=24, freq='MS')
uids = df_train['unique_id'].unique()

X_df = pd.DataFrame({
    'unique_id': [i for i in uids for _ in range(24)],
    'ds': list(future_dates) * len(uids)
})
X_df['MONTH'] = X_df['ds'].dt.month
X_df['is_summer'] = X_df['MONTH'].isin([5, 6, 7, 8]).astype(int)
X_df['is_holiday_season'] = X_df['MONTH'].isin([9, 10, 11, 12]).astype(int)
X_df['is_spring'] = X_df['MONTH'].isin([1, 2, 3, 4]).astype(int)
X_df['quarter'] = ((X_df['MONTH'] - 1) // 3) + 1
X_df['month_sin'] = np.sin(2 * np.pi * (X_df['MONTH'] - 1)/ 12)
X_df['month_cos'] = np.cos(2 * np.pi * (X_df['MONTH'] - 1)/ 12)
X_df['month_sq'] = X_df['MONTH'] ** 2
X_df['summer_peak'] = X_df['is_summer'] * X_df['MONTH']
X_df['holiday_peak'] = X_df['is_holiday_season'] * X_df['MONTH']

# Final Forecast Generation.
final_forecast3 = fcst.predict(h=24, X_df=X_df)
print("\nFinal 24-Month Forecast:")
print(final_forecast3.head())

Cross-Validation Results -> MAE: 39.905, MAPE: 17.68%
Global wMAPE: 14.06%
                                           unique_id        MAE     MAPE_%
0                                   Theft of Vehicle  58.514736  46.028742
1                                   Theft of Bicycle  31.476433  24.611589
2  Vehicle Collision or Pedestrian Struck (with I...  22.512476  21.646743
3                         Break and Enter Commercial  23.609226  14.632976
4                  Break and Enter Residential/Other  35.075310  12.889324
5                                           Mischief  49.880764  11.964813
6                                 Theft from Vehicle  89.787598  11.500016
7                           Offence Against a Person  27.027000   8.632383
8                                        Other Theft  21.261230   7.245229
                                           unique_id      wMAPE
4                                        Other Theft   7.318840
3                           Offence Against a P

In [46]:
import pandas as pd
import numpy as np
import xgboost as xgb
from mlforecast import MLForecast
from mlforecast.lag_transforms import RollingMean, RollingStd
from mlforecast.target_transforms import AutoDifferences, LocalStandardScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# 1. DATA PREPARATION
# Use the `df2_final_features` DataFrame, which contains all years and engineered features, for training.
df = df2_final_features.copy()

# Create a datetime column 'ds' from 'YEAR' and 'MONTH' for time series functionality.
df['ds'] = pd.to_datetime(df['YEAR'].astype(str) + '-' + df['MONTH'].astype(str) + '-01')
# Rename columns to 'unique_id' (for crime type) and 'y' (for crime count) as required by MLForecast.
df = df.rename(columns={'TYPE': 'unique_id', 'Monthly_Crime_Count_Type_wise': 'y'})

# Select the final training features, including 'unique_id', 'ds', 'y', and other engineered temporal features.
df_train = df[['unique_id', 'ds', 'y', 'MONTH', 'is_summer', 'is_holiday_season', 'is_spring', 'quarter', 'month_sin', 'month_cos', 'month_sq', 'summer_peak', 'holiday_peak']].copy()

# 2. DEFINE THE MODEL & FEATURES
# Initialize MLForecast with an XGBRegressor model, enabling categorical features and using 'hist' tree method for performance.
# 'freq' is set to 'MS' (Monthly Start) to specify the time series frequency.
# 'lags' are defined to capture past values as features.
# 'lag_transforms' apply rolling statistics (mean, std) over various windows to the lag features.
# 'date_features' tells the model to extract features from the date column (e.g., month of year).
# 'target_transforms' apply differencing to handle seasonality and trends in the target variable.
fcst = MLForecast(
    models=[xgb.XGBRegressor(enable_categorical=True, tree_method="hist", n_estimators=1200, learning_rate=0.02, random_state=42, verbosity=0)],
    freq='MS',                # Monthly Start frequency
    lags=[1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 18, 24],          # Lags chosen to capture various temporal dependencies
    lag_transforms={
        1: [RollingMean(window_size=3), RollingStd(window_size=3), RollingMean(window_size=6), RollingStd(window_size=6), RollingMean(window_size=12), RollingStd(window_size=12), RollingMean(window_size=24)] # Rolling statistics for trend and volatility
    },
    date_features=['month'],  # Use month as a feature
    target_transforms=[AutoDifferences(max_diffs=2)] # Apply auto-differencing
)

# 3. PERFORM CROSS-VALIDATION (The "Evaluation" Step)
# This simulates 3 separate 24-month forecasts from the historical data.
cv_results = fcst.cross_validation(
    df=df_train,
    h=24,
    n_windows=3,
    step_size=24,
    static_features=[] # Treats 'Month' as a dynamic variable
)

# Calculate Evaluation Metrics
# Define a function to calculate Mean Absolute Error (MAE) and Mean Absolute Percentage Error (MAPE).
def calculate_metrics(df_cv, model_name):
    actual = df_cv['y']
    pred = df_cv[model_name]
    mae = mean_absolute_error(actual, pred)
    mape = mean_absolute_percentage_error(actual, pred)*100
    return mae, mape

# Calculate and print global MAE and MAPE from cross-validation results.
mae, mape = calculate_metrics(cv_results, 'XGBRegressor')
print(f"Cross-Validation Results -> MAE: {mae:.3f}, MAPE: {mape:.2f}%")

import numpy as np

# Define a function to calculate global Weighted Mean Absolute Percentage Error (WMAPE).
def get_wmape(df_cv, model_col='XGBRegressor'):
    # Sum of all absolute errors across all months and crime types.
    total_abs_error = np.sum(np.abs(df_cv['y'] - df_cv[model_col]))

    # Sum of all actual crime counts.
    total_actuals = np.sum(df_cv['y'])

    # Calculate weighted percentage.
    wmape = (total_abs_error / total_actuals) * 100
    return wmape

# Usage
score = get_wmape(cv_results)
print(f"Global wMAPE: {score:.2f}%")


#4. Calculating MAE, MAPE per crime type

def per_crime_type_metrics(df, model_col='XGBRegressor'):
    """
    Calculates MAE and MAPE per unique_id (crime type)
    using sklearn metric functions.
    """

    results = []

    grouped = df.groupby('unique_id')

    for uid, group in grouped:
        actual = group['y']
        pred = group[model_col]
        mae = mean_absolute_error(actual, pred)
        mape = mean_absolute_percentage_error(actual, pred) * 100

        results.append({
            'unique_id': uid,
            'MAE': mae,
            'MAPE_%': mape
        })

    result_df = pd.DataFrame(results)
    return result_df.sort_values('MAPE_%', ascending=False).reset_index(drop=True)

#Usage

metrics_per_type = per_crime_type_metrics(cv_results, model_col='XGBRegressor')
print(metrics_per_type)

#Calculating WMAPE per crime type

def per_id_wmape(df_cv, model_col='XGBRegressor'):
    # Group by unique_id and calculate (sum of errors / sum of actuals).
    summary = df_cv.groupby('unique_id').apply(
        lambda x: (np.sum(np.abs(x['y'] - x[model_col])) / np.sum(x['y'])) * 100
    ).reset_index()

    summary.columns = ['unique_id', 'wMAPE']
    return summary

# Usage
type_analysis = per_id_wmape(cv_results)
print(type_analysis.sort_values('wMAPE'))


# 5. FINAL FIT & FUTURE PREDICTION
# After verifying performance, train on ALL data and project 2 years ahead.
fcst.fit(df_train, static_features=[], max_horizon=24)

# Create future dynamic features (X_df) for the next 24 months.
last_date = df_train['ds'].max()
future_dates = pd.date_range(start=last_date + pd.offsets.MonthBegin(1), periods=24, freq='MS')
uids = df_train['unique_id'].unique()

X_df = pd.DataFrame({
    'unique_id': [i for i in uids for _ in range(24)],
    'ds': list(future_dates) * len(uids)
})
X_df['MONTH'] = X_df['ds'].dt.month
X_df['is_summer'] = X_df['MONTH'].isin([5, 6, 7, 8]).astype(int)
X_df['is_holiday_season'] = X_df['MONTH'].isin([9, 10, 11, 12]).astype(int)
X_df['is_spring'] = X_df['MONTH'].isin([1, 2, 3, 4]).astype(int)
X_df['quarter'] = ((X_df['MONTH'] - 1) // 3) + 1
X_df['month_sin'] = np.sin(2 * np.pi * (X_df['MONTH'] - 1)/ 12)
X_df['month_cos'] = np.cos(2 * np.pi * (X_df['MONTH'] - 1)/ 12)
X_df['month_sq'] = X_df['MONTH'] ** 2
X_df['summer_peak'] = X_df['is_summer'] * X_df['MONTH']
X_df['holiday_peak'] = X_df['is_holiday_season'] * X_df['MONTH']

# Final Forecast Generation.
final_forecast4 = fcst.predict(h=24, X_df=X_df)
print("\nFinal 24-Month Forecast:")
print(final_forecast4.head())

Cross-Validation Results -> MAE: 43.458, MAPE: 16.65%
Global wMAPE: 15.36%
                                           unique_id         MAE     MAPE_%
0                                   Theft of Vehicle   32.512592  32.073790
1                                   Theft of Bicycle   50.398830  25.493348
2  Vehicle Collision or Pedestrian Struck (with I...   20.022879  16.355191
3                         Break and Enter Commercial   28.240322  16.348529
4                                 Theft from Vehicle  116.870049  14.374639
5                           Offence Against a Person   36.384930  12.937056
6                  Break and Enter Residential/Other   31.848507  12.337593
7                                           Mischief   44.459682  11.069114
8                                        Other Theft   30.386292   8.859321
                                           unique_id      wMAPE
4                                        Other Theft   9.511817
1                  Break and Enter Re

In [47]:
# Create a copy of `final_forecast4` for post-processing.
final_forecast5 = final_forecast4.copy()
# Rename columns for clarity, changing 'unique_id' to 'TYPE', 'ds' to 'Date', and the model's prediction column to 'Monthly_Crime_Count_Type_wise'.
final_forecast5.rename(columns={'unique_id':'TYPE', 'ds':'Date', 'XGBRegressor': 'Monthly_Crime_Count_Type_wise'}, inplace=True)
# Extract the year from the 'Date' column and create a new 'YEAR' column.
final_forecast5['YEAR'] = final_forecast5['Date'].dt.year
# Extract the month from the 'Date' column and create a new 'MONTH' column.
final_forecast5['MONTH'] = final_forecast5['Date'].dt.month
# Select and reorder the relevant columns for the final output.
final_forecast5 = final_forecast5[['TYPE', 'YEAR', 'MONTH', 'Monthly_Crime_Count_Type_wise']]
# Sort the DataFrame by crime type, year, and month to ensure chronological order for each type.
final_forecast5.sort_values(by=['TYPE', 'YEAR', 'MONTH'], inplace=True)
# Reset the index of the DataFrame after sorting.
final_forecast5.reset_index(drop=True, inplace=True)
# Display the processed forecast DataFrame.
final_forecast5

,TYPE,YEAR,MONTH,Monthly_Crime_Count_Type_wise
0,Break and Enter Commercial,2012,1,219.667725
1,Break and Enter Commercial,2012,2,173.128967
2,Break and Enter Commercial,2012,3,196.674988
3,Break and Enter Commercial,2012,4,193.611542
4,Break and Enter Commercial,2012,5,195.368332
...,...,...,...,...
211,Vehicle Collision or Pedestrian Struck (with I...,2013,8,246.165268
212,Vehicle Collision or Pedestrian Struck (with I...,2013,9,249.691574
213,Vehicle Collision or Pedestrian Struck (with I...,2013,10,240.467102
214,Vehicle Collision or Pedestrian Struck (with I...,2013,11,231.155090


# *Saving the XGBoost Forecasted Data for Streamlit Visualisations*

In [ ]:
# Define a function to convert numerical month values (1-12) to their abbreviated names.
def get_month_name(month_num):
  if month_num == 1:
    return 'Jan'
  elif month_num == 2:
    return 'Feb'
  elif month_num == 3:
    return 'Mar'
  elif month_num == 4:
    return 'Apr'
  elif month_num == 5:
    return 'May'
  elif month_num == 6:
    return 'Jun'
  elif month_num == 7:
    return 'Jul'
  elif month_num == 8:
    return 'Aug'
  elif month_num == 9:
    return 'Sep'
  elif month_num == 10:
    return 'Oct'
  elif month_num == 11:
    return 'Nov'
  else:
    return 'Dec'

# Create a copy of `final_forecast4` for further processing and visualization.
final_forecast8 = final_forecast4.copy()
# Rename columns for clarity, changing 'unique_id' to 'TYPE', 'ds' to 'Date', and the model's prediction column to 'Crime_Count'.
final_forecast8.rename(columns={'unique_id':'TYPE', 'ds':'Date', 'XGBRegressor': 'Crime_Count'}, inplace=True)
# Extract the year from the 'Date' column and create a new 'YEAR' column.
final_forecast8['YEAR'] = final_forecast8['Date'].dt.year
# Extract the month from the 'Date' column and create a new 'MONTH' column.
final_forecast8['MONTH'] = final_forecast8['Date'].dt.month
# Initialize 'Month_name' column with NaN values.
final_forecast8['Month_name'] = np.nan
# Apply the `get_month_name` function to populate the 'Month_name' column based on 'MONTH'.
final_forecast8['Month_name'] = final_forecast8['MONTH'].apply(get_month_name)
# Select and reorder the relevant columns for the final output.
final_forecast8 = final_forecast8[['TYPE', 'YEAR', 'MONTH', 'Month_name', 'Crime_Count']]
# Sort the DataFrame by crime type, year, month, and month name for consistent ordering.
final_forecast8.sort_values(by=['TYPE', 'YEAR', 'MONTH', 'Month_name'], inplace=True)
# Reset the index of the DataFrame after sorting.
final_forecast8.reset_index(drop=True, inplace=True)

# Group the original `Train_df` to get past crime counts by type, year, month, and month name.
Past_df_type_wise_crime_count = Train_df.groupby(['TYPE', 'YEAR', 'MONTH', 'Month_name']).size().reset_index(name='Crime_Count')
# Apply the `get_month_name` function to ensure consistent month names in the past data.
Past_df_type_wise_crime_count['Month_name'] = Past_df_type_wise_crime_count['MONTH'].apply(get_month_name)
# Sort the past crime count DataFrame for consistent ordering.
Past_df_type_wise_crime_count.sort_values(by=['TYPE', 'YEAR', 'MONTH', 'Month_name'], ascending=[True, True, True, True], inplace=True)
# Reset the index.
Past_df_type_wise_crime_count.reset_index(drop=True, inplace=True)

# Concatenate the past crime data with the LightGBM forecasted data (Note: this should probably be `final_forecast8` for XGBoost).
# This creates a single DataFrame containing both historical and predicted crime counts.
# There seems to be a copy-paste error here, `final_forecast7` should be `final_forecast8` to merge XGBoost data.
Past_and_Future_df_type_wise_crime_count_XGBoost = pd.concat([Past_df_type_wise_crime_count, final_forecast7], ignore_index=True, axis=0)
# Display the combined DataFrame.
Past_and_Future_df_type_wise_crime_count_XGBoost

,TYPE,YEAR,MONTH,Month_name,Crime_Count
0,Break and Enter Commercial,1999,1,Jan,303.000000
1,Break and Enter Commercial,1999,2,Feb,254.000000
2,Break and Enter Commercial,1999,3,Mar,292.000000
3,Break and Enter Commercial,1999,4,Apr,266.000000
4,Break and Enter Commercial,1999,5,May,291.000000
...,...,...,...,...,...
1615,Vehicle Collision or Pedestrian Struck (with I...,2013,8,Aug,155.484039
1616,Vehicle Collision or Pedestrian Struck (with I...,2013,9,Sep,167.898621
1617,Vehicle Collision or Pedestrian Struck (with I...,2013,10,Oct,155.274078
1618,Vehicle Collision or Pedestrian Struck (with I...,2013,11,Nov,130.742920


In [ ]:
from google.colab import files
# Save the combined historical and XGBoost forecasted DataFrame to a CSV file.
Past_and_Future_df_type_wise_crime_count_XGBoost.to_csv('Past_and_Future_df_type_wise_crime_count_XGBoost.csv', index=False)
# Initiate the download of the CSV file to the local machine.
files.download('Past_and_Future_df_type_wise_crime_count_XGBoost.csv')

# *Visualisations Based on the Forecast of the XGBoost Regressor Model*

In [48]:
# Create a copy of `final_forecast4` for processing.
final_forecast5 = final_forecast4.copy()
# Rename columns for clarity.
final_forecast5.rename(columns={'unique_id':'TYPE', 'ds':'Date', 'XGBRegressor': 'Monthly_Crime_Count_Type_wise'}, inplace=True)
# Extract year and month.
final_forecast5['YEAR'] = final_forecast5['Date'].dt.year
final_forecast5['MONTH'] = final_forecast5['Date'].dt.month
# Select and reorder columns.
final_forecast5 = final_forecast5[['TYPE', 'YEAR', 'MONTH', 'Monthly_Crime_Count_Type_wise']]
# Sort the DataFrame.
final_forecast5.sort_values(by=['TYPE', 'YEAR', 'MONTH'], inplace=True)
# Reset the index.
final_forecast5.reset_index(drop=True, inplace=True)

# Group the forecasted data by 'TYPE' and 'YEAR' to get total yearly crime counts for the future.
Yearly_type_wise_crime_count_future = final_forecast5.groupby(['TYPE', 'YEAR'])['Monthly_Crime_Count_Type_wise'].sum().reset_index(name='Crime Count')
# Sort the future crime count DataFrame.
Yearly_type_wise_crime_count_future.sort_values(by=['TYPE', 'YEAR'], ascending=[True, True], inplace=True)
# Reset the index.
Yearly_type_wise_crime_count_future.reset_index(drop=True, inplace=True)

# Group the original `Train_df` by 'TYPE' and 'YEAR' to get total yearly crime counts for the past.
Yearly_type_wise_crime_count = Train_df.groupby(['TYPE', 'YEAR']).size().reset_index(name='Crime Count')
# Sort the past crime count DataFrame.
Yearly_type_wise_crime_count.sort_values(by=['TYPE', 'YEAR'], ascending=[True, True], inplace=True)
# Reset the index.
Yearly_type_wise_crime_count.reset_index(drop=True, inplace=True)

# Concatenate past and future yearly crime counts.
df = pd.concat([Yearly_type_wise_crime_count, Yearly_type_wise_crime_count_future], ignore_index=True, axis=0)

# Create a line plot to visualize type-wise crime counts over past and future years (XGBoost forecast).
fig = px.line(df, x='YEAR', y='Crime Count', color='TYPE', title='Type Wise Crime Count Over the Past Years & the Future Years(Forecasted Using XGBoost Regressor)', labels={'YEAR':'Year'})
# Update plot layout for aesthetics.
fig.update_layout(xaxis_title='Year', yaxis_title='Crime Count', title_x = 0.5, title_font_size=24, title_font_family='Arial Black', width=1400, height=700)
# Display the plot.
fig.show()

***`Trend Observed:`***

Forecast continues existing trend patterns.

***`Key Insight:`***

XGBoost effectively models long-term crime trends.

In [49]:
# Create a copy of `final_forecast5` for processing.
df1 = final_forecast5.copy()
# Initialize 'Month_name' column with NaN values.
df1['Month_name'] = np.nan

# Define a function to get full month names.
def get_month_name(month_num):
  if month_num == 1:
    return 'January'
  elif month_num == 2:
    return 'February'
  elif month_num == 3:
    return 'March'
  elif month_num == 4:
    return 'April'
  elif month_num == 5:
    return 'May'
  elif month_num == 6:
    return 'June'
  elif month_num == 7:
    return 'July'
  elif month_num == 8:
    return 'August'
  elif month_num == 9:
    return 'September'
  elif month_num == 10:
    return 'October'
  elif month_num == 11:
    return 'November'
  else:
    return 'December'

# Apply the function to populate 'Month_name' in the future data.
df1['Month_name'] = df1['MONTH'].apply(get_month_name)
# Rename 'Monthly_Crime_Count_Type_wise' to 'Crime_Count'.
df1.rename(columns={'Monthly_Crime_Count_Type_wise':'Crime_Count'}, inplace=True)
# Sort the future monthly data.
df1.sort_values(by=['TYPE', 'YEAR', 'MONTH', 'Month_name'], inplace=True)
# Reset the index.
df1.reset_index(drop=True, inplace=True)


# Group the original `Train_df` to get past monthly crime counts by type, year, month, and month name.
df2 = Train_df.groupby(['TYPE', 'YEAR', 'MONTH', 'Month_name']).size().reset_index(name='Crime_Count')
# Rename columns for consistency.
df2.columns=['TYPE', 'YEAR', 'MONTH', 'Month_name', 'Crime_Count']
# Sort the past monthly data.
df2.sort_values(by=['TYPE', 'YEAR', 'MONTH', 'Month_name'], ascending=[True, True, True, True], inplace=True)
# Reset the index.
df2.reset_index(drop=True, inplace=True)

# Concatenate past and future monthly crime counts.
df = pd.concat([df2, df1], ignore_index=True, axis=0)

# Create a faceted line plot to visualize monthly crime counts for past and future years (XGBoost forecast).
# Each facet represents a crime type, and lines are colored by year.
fig = px.line(df, x='Month_name', y='Crime_Count', color='YEAR', title='Crime Count Peak Months of Past Years & the Future Years(Forecasted Using XGBoost Regressor)', facet_col='TYPE', facet_col_wrap=3,
             labels={'MONTH': 'Month', 'Crime_Count': 'Crime Count'})
# Update plot layout for aesthetics.
fig.update_layout(xaxis_title='Month', yaxis_title='Crime Count', title_x=0.5, title_font_size=24, title_font_family='Arial Black', width=1400, height=1200)
# Display the plot.
fig.show()

***`Trend Observed:`***

Seasonal peaks and troughs preserved.

# *`Comparative Model Conclusion(LightGBM Vs XGBoost):`*

In this project, both LightGBM and XGBoost were implemented to forecast monthly crime counts across different crime types. Both models demonstrated strong predictive performance by successfully capturing seasonal patterns, lag dependencies, and nonlinear relationships in the historical crime data.

However, comparative evaluation revealed that LightGBM slightly outperformed XGBoost in terms of forecast stability, computational efficiency, and consistency across crime categories. LightGBM produced smoother predictions and handled feature-engineered time-series inputs more efficiently, making it particularly suitable for multi-series monthly crime forecasting.

XGBoost also delivered accurate and reliable forecasts, though it showed marginally higher sensitivity to short-term fluctuations and required slightly more computational resources.

Overall, LightGBM emerged as the most effective model for high-frequency crime forecasting in this project, while XGBoost remains a strong and dependable alternative. The successful application of both models demonstrates the effectiveness of gradient boosting techniques in modeling complex temporal crime patterns and supporting data-driven decision-making.

# **SARIMAX Model**

# *`Model Description:`*
***SARIMAX (Seasonal AutoRegressive Integrated Moving Average with eXogenous Variables)***

SARIMAX is an extension of the ARIMA (AutoRegressive Integrated Moving Average) model that incorporates both seasonality and external explanatory variables (exogenous variables) into time series forecasting. It is a statistical modeling approach specifically designed for structured temporal data.

*The model combines four major components:*

*Autoregressive (AR) – Uses past values of the time series to predict future values.

*Integrated (I) – Applies differencing to make the time series stationary.

*Moving Average (MA) – Uses past forecast errors to improve predictions.

*Seasonal Component (S) – Captures repeating seasonal patterns over fixed intervals.

The “X” in SARIMAX allows the inclusion of external variables that may influence the target time series.

# *`Key Characteristics of SARIMAX:`*

Designed specifically for time series forecasting

Captures both trend and seasonal components

Produces interpretable statistical parameters

Effective for structured and relatively stable time-series data

Suitable for medium- to long-term forecasting

# *`Why SARIMAX Was Used in This Project:`*

In this project, SARIMAX was used to forecast yearly crime counts for each neighbourhood.

It was selected because:

Yearly crime data exhibits long-term trends and structured temporal behavior.

Aggregated yearly data is smoother and more suitable for statistical time-series modeling.

Neighbourhood-level forecasting benefits from interpretable trend modeling.

The model effectively captures autoregressive dependencies in yearly crime counts.

# *Model Training & Forecast*

In [15]:
# Group the `Train_df` by 'NEIGHBOURHOOD' and 'YEAR' to get the total yearly crime count for each neighborhood.
Train_df_3 = Train_df.groupby(['NEIGHBOURHOOD', 'YEAR']).size().reset_index(name='Total_Yearly_Crime_Count_per_Neighbourhood')
# Sort the DataFrame by neighborhood and year for chronological and grouped display.
Train_df_3.sort_values(by=['NEIGHBOURHOOD', 'YEAR'], ascending=True, inplace=True)
# Reset the index.
Train_df_3.reset_index(drop=True, inplace=True)
# Display the resulting DataFrame.
Train_df_3

,NEIGHBOURHOOD,YEAR,Total_Yearly_Crime_Count_per_Neighbourhood
0,Arbutus Ridge,1999,616
1,Arbutus Ridge,2000,648
2,Arbutus Ridge,2001,619
3,Arbutus Ridge,2002,441
4,Arbutus Ridge,2003,361
...,...,...,...
320,West Point Grey,2007,317
321,West Point Grey,2008,311
322,West Point Grey,2009,279
323,West Point Grey,2010,365


In [16]:
# Filter the `Train_df_3` DataFrame to show only the crime counts for the 'Central Business District' neighborhood.
# This allows for a quick inspection of yearly trends for a specific neighborhood.
Train_df_3[Train_df_3['NEIGHBOURHOOD'] == 'Central Business District']

,NEIGHBOURHOOD,YEAR,Total_Yearly_Crime_Count_per_Neighbourhood
13,Central Business District,1999,9815
14,Central Business District,2000,8744
15,Central Business District,2001,7668
16,Central Business District,2002,8480
17,Central Business District,2003,8251
18,Central Business District,2004,8209
19,Central Business District,2005,6893
20,Central Business District,2006,5831
21,Central Business District,2007,5722
22,Central Business District,2008,5409


In [ ]:
# Save the `Train_df_3` DataFrame (yearly crime counts per neighborhood) to a CSV file.
# This is done to make the dataset easily accessible for external tools or for further analysis, particularly for SARIMAX model training in other environments like VS Code.
Train_df_SARIMAX_Model = Train_df_3.copy()
Train_df_SARIMAX_Model.to_csv('Train_df_SARIMAX_Model.csv', index=False)
from google.colab import files
files.download('Train_df_SARIMAX_Model.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

# IMPORTING LIBRARIES

import numpy as np
import warnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

warnings.filterwarnings("ignore")

# LOADING & PREPARING THE DATA

# Creating a copy of Train_df_3, which contains yearly crime counts per neighborhood.
df = Train_df_3.copy()

# Renaming columns to 'unique_id' (for neighborhood), 'y' (for crime count), and 'year'.
df = df.rename(columns={
    "NEIGHBOURHOOD": "unique_id",
    "Total_Yearly_Crime_Count_per_Neighbourhood": "y",
    "YEAR": "year"
})

# Creating a datetime column 'ds' from the 'year' for time series processing. Dates are set to January 1st.
df["ds"] = pd.to_datetime(df["year"].astype(str) + "-01-01")

# Sort the DataFrame by 'unique_id' and 'ds' to ensure correct time series order.
df = df.sort_values(["unique_id", "ds"]).reset_index(drop=True)


# SARIMA WALK-FORWARD CROSS VALIDATION

# Define a function for walk-forward cross-validation with SARIMAX.
def sarima_cross_validation(data, h=2, n_windows=3,
                            order=(1,1,1),
                            seasonal_order=(0,0,0,0)):
    """
    Performs walk-forward cross-validation for SARIMAX models.
    h = forecast horizon (e.g., 2 years)
    n_windows = number of rolling splits
    order = (p, d, q) for ARIMA
    seasonal_order = (P, D, Q, S) for seasonal ARIMA
    """

    results = []
    neighbourhoods = data["unique_id"].unique()

    for uid in neighbourhoods:

        # Filtering data for the current neighborhood.
        sub = data[data["unique_id"] == uid].copy()
        sub = sub.set_index("ds") # Set 'ds' as index for time series modeling.
        y = sub["y"]

        # Skiping neighborhoods with insufficient data for cross-validation.
        if len(y) < h + n_windows:
            continue

        # Iterating through the defined number of windows for cross-validation.
        for i in range(n_windows):

            # Determining the split point for training and testing data.
            split_point = len(y) - h*(n_windows - i)

            # Splitting data into training and testing sets.
            train = y.iloc[:split_point]
            test = y.iloc[split_point:split_point+h]

            try:
                # Initializing and fitting the SARIMAX model.
                model = SARIMAX(
                    train,
                    order=order,
                    seasonal_order=seasonal_order,
                    enforce_stationarity=False, # Allow non-stationary data.
                    enforce_invertibility=False # Allow non-invertible models.
                )

                fitted = model.fit(disp=False) # Fit model without verbose output.

                # Generating forecast for the test horizon.
                forecast = fitted.forecast(steps=h)

                # Storing results in a temporary DataFrame.
                temp = pd.DataFrame({
                    "unique_id": uid,
                    "ds": test.index,
                    "y": test.values,
                    "prediction": forecast.values
                })

                results.append(temp)

            except:
                # Continue if model fitting or forecasting fails for a particular split.
                continue

    # Concatenate all cross-validation results.
    return pd.concat(results).reset_index(drop=True)


# Running cross-validation for the SARIMA model with specified parameters.
cv_results = sarima_cross_validation(
    df,
    h=2, # Forecast horizon of 2 years.
    n_windows=3, # 3 cross-validation windows.
    order=(1,1,1), # ARIMA order (p,d,q).
    seasonal_order=(0,0,0,0) # No seasonal component specified here.
)


# GLOBAL METRICS

# Calculating global Mean Absolute Error (MAE) from cross-validation results.
global_mae = mean_absolute_error(cv_results["y"], cv_results["prediction"])
# Calculating global Mean Absolute Percentage Error (MAPE).
global_mape = mean_absolute_percentage_error(cv_results["y"], cv_results["prediction"]) * 100

print(f"\nGlobal MAE: {global_mae:.3f}")
print(f"Global MAPE: {global_mape:.2f}%")

# GLOBAL WMAPE

# Defined a function to calculate global Weighted Mean Absolute Percentage Error (WMAPE).
def global_wmape(df):
    return (np.sum(np.abs(df["y"] - df["prediction"])) /
            np.sum(df["y"])) * 100

# Calculating and print global WMAPE.
print(f"Global wMAPE: {global_wmape(cv_results):.2f}%")

# PER-NEIGHBOURHOOD METRICS

# Defined a function to calculate MAE and MAPE for each neighborhood.
def per_neighbourhood_metrics(df):

    results = []

    grouped = df.groupby("unique_id")

    for uid, group in grouped:

        mae = mean_absolute_error(group["y"], group["prediction"])
        mape = mean_absolute_percentage_error(group["y"], group["prediction"]) * 100

        results.append({
            "unique_id": uid,
            "MAE": mae,
            "MAPE_%": mape
        })

    return pd.DataFrame(results).sort_values("MAPE_%", ascending=False)


print("\nPer-Neighbourhood Metrics:")
print(per_neighbourhood_metrics(cv_results))

# FINAL MODEL (TRAIN ON FULL DATA)
# FORECAST 2012–2013

# Defined a function to train the final SARIMAX model on all available data and generate future forecasts.
def final_forecast(data, forecast_years=2,
                   order=(1,1,1),
                   seasonal_order=(0,0,0,0)):

    forecasts = []

    neighbourhoods = data["unique_id"].unique()

    for uid in neighbourhoods:

        # Filtering data for the current neighborhood.
        sub = data[data["unique_id"] == uid].copy()
        sub = sub.set_index("ds") # Set 'ds' as index.
        y = sub["y"]

        # Initializing and fitting the SARIMAX model on the full historical data for the neighborhood.
        model = SARIMAX(
            y,
            order=order,
            seasonal_order=seasonal_order,
            enforce_stationarity=False,
            enforce_invertibility=False
        )

        fitted = model.fit(disp=False)

        # Generating the forecast for the specified number of future years.
        forecast = fitted.forecast(steps=forecast_years)

        # Storing the forecast results.
        temp = pd.DataFrame({
            "unique_id": uid,
            "ds": forecast.index,
            "prediction": forecast.values
        })

        forecasts.append(temp)

    # Concatenating all neighborhood forecasts.
    return pd.concat(forecasts).reset_index(drop=True)


# Generating final predictions for 2012-2013 using the full dataset.
final_predictions = final_forecast(df, forecast_years=2)

print("\nForecast for 2012–2013:")
print(final_predictions)


Global MAE: 176.409
Global MAPE: 17.74%
Global wMAPE: 14.43%

Per-Neighbourhood Metrics:
                    unique_id         MAE     MAPE_%
12                   Musqueam   10.880403  39.306688
19               Stanley Park   76.601014  37.070555
7                  Kerrisdale  125.461617  29.431075
2           Dunbar-Southlands  139.647310  29.270070
3                    Fairview  493.627743  28.442548
0               Arbutus Ridge   77.431755  22.467713
18               South Cambie   79.057525  21.670861
24            West Point Grey   66.965604  19.816147
17                Shaughnessy   64.425181  18.996820
20                 Strathcona  242.930290  18.626442
21                     Sunset  154.910811  17.584036
8                   Killarney  101.078014  17.478282
23                   West End  372.263457  16.665043
22        Victoria-Fraserview   84.663255  14.931756
10                    Marpole  103.734257  14.507651
1   Central Business District  924.213467  13.499465
11       

In [ ]:
final_predictions1 = final_predictions.copy()
# Rename columns for clarity, changing 'unique_id' to 'NEIGHBOURHOOD', 'ds' to 'Date', and 'prediction' to 'Crime Count'.
final_predictions1.rename(columns={'unique_id':'NEIGHBOURHOOD', 'ds':'Date', 'prediction':'Crime Count'}, inplace=True)
# Extract the year from the 'Date' column and create a new 'YEAR' column.
final_predictions1['YEAR'] = final_predictions1['Date'].dt.year

# Sort the DataFrame by 'NEIGHBOURHOOD' and 'YEAR' for consistent ordering.
final_predictions1.sort_values(by=['NEIGHBOURHOOD', 'YEAR'], inplace=True)
# Reset the index of the DataFrame after sorting.
final_predictions1.reset_index(drop=True, inplace=True)
# Display the processed forecast DataFrame.
final_predictions1

,NEIGHBOURHOOD,Date,Crime Count,YEAR
0,Arbutus Ridge,2012-01-01,357.698047,2012
1,Arbutus Ridge,2013-01-01,352.977149,2013
2,Central Business District,2012-01-01,8404.552603,2012
3,Central Business District,2013-01-01,8484.359598,2013
4,Dunbar-Southlands,2012-01-01,582.526135,2012
5,Dunbar-Southlands,2013-01-01,586.661213,2013
6,Fairview,2012-01-01,1937.802740,2012
7,Fairview,2013-01-01,1897.437433,2013
8,Grandview-Woodland,2012-01-01,1600.565780,2012
9,Grandview-Woodland,2013-01-01,1567.793281,2013


# **AutoARIMA + StatsForecast**

In [17]:
!pip install statsforecast

In [20]:

# IMPORTING LIBRARIES

import pandas as pd
import numpy as np
import warnings
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

warnings.filterwarnings("ignore")


# LOADING & PREPARING THE DATA

# Creating a copy of Train_df_3, which contains yearly crime counts per neighborhood.
df = Train_df_3.copy()

# Renaming columns to 'unique_id' (for neighborhood), 'y' (for crime count), and 'ds'.
df = df.rename(columns={
    "NEIGHBOURHOOD": "unique_id",
    "Total_Yearly_Crime_Count_per_Neighbourhood": "y",
    "YEAR": "ds"
})

# Creating a datetime column 'ds' from the 'year' for time series processing. Dates are set to January 1st.
df["ds"] = pd.to_datetime(df["ds"], format='%Y')

# Sort the DataFrame by 'unique_id' and 'ds' to ensure correct time series order.
df = df.sort_values(["unique_id", "ds"]).reset_index(drop=True)


# AutoARIMA WALK-FORWARD CROSS VALIDATION


# Initializing StatsForecast *once* with the model instance
sf_cv = StatsForecast(
    models=[AutoARIMA(approximation=False, stepwise=True)],
    freq='YS', # Yearly start frequency
    n_jobs=-1
)

# Performing cross-validation using StatsForecast's built-in method
cv_results = sf_cv.cross_validation(
    df=df, # Use the full prepared DataFrame
    h=2, # Forecast horizon of 2 years.
    n_windows=3, # 3 cross-validation windows.
    step_size=1, # Move training window forward by 1 year for each window.
    time_col='ds',
    target_col='y',
    id_col='unique_id'
)


# GLOBAL METRICS

# Calculating global Mean Absolute Error (MAE) from cross-validation results.
global_mae = mean_absolute_error(cv_results["y"], cv_results["AutoARIMA"])
# Calculating global Mean Absolute Percentage Error (MAPE).
global_mape = mean_absolute_percentage_error(cv_results["y"], cv_results["AutoARIMA"]) * 100

print(f"\nGlobal MAE: {global_mae:.3f}")
print(f"Global MAPE: {global_mape:.2f}%")


# GLOBAL WMAPE

# Defined a function to calculate global Weighted Mean Absolute Percentage Error (WMAPE).
def global_wmape(df_cv, model_col='AutoARIMA'):
    return (np.sum(np.abs(df_cv["y"] - df_cv[model_col])) /
            np.sum(df_cv["y"])) * 100

# Calculated and printed the global WMAPE.
print(f"Global wMAPE: {global_wmape(cv_results):.2f}%")


# PER-NEIGHBOURHOOD METRICS

# Defined a function to calculate MAE and MAPE for each neighborhood.
def per_neighbourhood_metrics(df_cv, model_col='AutoARIMA'):

    results = []

    grouped = df_cv.groupby("unique_id")

    for uid, group in grouped:

        mae = mean_absolute_error(group["y"], group[model_col])
        mape = mean_absolute_percentage_error(group["y"], group[model_col]) * 100

        results.append({
            "unique_id": uid,
            "MAE": mae,
            "MAPE_%": mape
        })

    return pd.DataFrame(results).sort_values("MAPE_%", ascending=False)


print("\nPer-Neighbourhood Metrics:")
print(per_neighbourhood_metrics(cv_results))

# FINAL MODEL (TRAIN ON FULL DATA)
# FORECAST 2012–2013

# Initializing StatsForecast for the final forecast
sf_final = StatsForecast(
    models=[AutoARIMA(approximation=False, stepwise=False)],
    freq='YS',
    n_jobs=-1
)

# Fit the model on the entire dataset (df) and generate future forecasts.
sf_final.fit(df, time_col='ds', target_col='y', id_col='unique_id')
final_predictions = sf_final.predict(h=2) # Forecast 2 years (2012-2013)

print("\nForecast for 2012–2013:")
print(final_predictions)



Global MAE: 250.991
Global MAPE: 22.87%
Global wMAPE: 20.15%

Per-Neighbourhood Metrics:
                    unique_id          MAE     MAPE_%
12                   Musqueam    21.535761  88.012856
7                  Kerrisdale   174.833328  43.218768
2           Dunbar-Southlands   167.333328  37.928513
23                   West End   745.736145  32.526758
4          Grandview-Woodland   484.340240  30.655435
3                    Fairview   534.674988  28.516173
19               Stanley Park    59.833332  25.083780
9                   Kitsilano   314.079163  21.696508
20                 Strathcona   285.629608  20.788956
18               South Cambie    82.500000  20.343019
5            Hastings-Sunrise   191.899979  19.923520
17                Shaughnessy    62.166668  19.031097
6    Kensington-Cedar Cottage   288.500000  18.242711
1   Central Business District  1261.111206  17.917065
21                     Sunset   153.942596  17.432004
22        Victoria-Fraserview    92.500000  16

# *SARIMAX Vs AutoARIMA*

While AutoARIMA provides a robust automated baseline, the 13-year horizon for FBI crime data proved too short for the AIC-based(Akaike Information Criterion) penalty system of the automated search. The manually configured SARIMAX model was better able to capture the specific growth trends and local variance of the 24 neighbourhoods, resulting in a 29.7% reduction in Mean Absolute Error (MAE) compared to the automated baseline.

So finally we can describe the AIC as the "Mathematical Judge" that AutoARIMA uses to prevent overfitting. In our report, since we had a small dataset of 13 years, the AIC penalty was likely too aggressive, which is why our manually tuned SARIMAX outperformed the AIC-optimized AutoARIMA.

# *Saving the SARIMAX Model Forecasted Data for Streamlit Visualisations*

In [ ]:
final_predictions2 = final_predictions.copy()
# Rename columns for clarity and consistency.
final_predictions2.rename(columns={'unique_id':'NEIGHBOURHOOD', 'ds':'Date', 'prediction':'Crime_Count'}, inplace=True)
# Drop the 'Date' column as 'YEAR' will be used.
final_predictions2.drop(columns=['Date'], inplace=True)
# Extract the year from the original `final_predictions1` Date column (assuming `final_predictions1` is already processed).
final_predictions2['YEAR'] = final_predictions1['Date'].dt.year
# Pop the 'YEAR' column and insert it as the second column for consistent ordering.
col_to_move = final_predictions2.pop('YEAR')
final_predictions2.insert(1, 'YEAR', col_to_move)

# Sort the DataFrame by 'NEIGHBOURHOOD' and 'YEAR' for consistent ordering.
final_predictions2.sort_values(by=['NEIGHBOURHOOD', 'YEAR'], inplace=True)
# Reset the index.
final_predictions2.reset_index(drop=True, inplace=True)
# Display the processed DataFrame.
final_predictions2

# Group the original `Train_df` to get past yearly crime counts per neighborhood.
Yearly_neighbourhood_crime_count_past = Train_df.groupby(['NEIGHBOURHOOD', 'YEAR']).size().reset_index(name='Crime_Count')
# Sort the past crime count DataFrame.
Yearly_neighbourhood_crime_count_past.sort_values(by=['NEIGHBOURHOOD', 'YEAR'], ascending=[True, True], inplace=True)
# Reset the index.
Yearly_neighbourhood_crime_count_past.reset_index(drop=True, inplace=True)

# Concatenate the past crime data with the SARIMAX forecasted data.
Past_and_Future_df_yearly_neighbourhood_crime_count_sarimax = pd.concat([Yearly_neighbourhood_crime_count_past, final_predictions2], ignore_index=True, axis=0)
# Sort the combined DataFrame for consistent ordering.
Past_and_Future_df_yearly_neighbourhood_crime_count_sarimax.sort_values(by=['NEIGHBOURHOOD', 'YEAR'], ascending=[True, True], inplace=True)
# Reset the index.
Past_and_Future_df_yearly_neighbourhood_crime_count_sarimax.reset_index(drop=True, inplace=True)
# Display the combined DataFrame.
Past_and_Future_df_yearly_neighbourhood_crime_count_sarimax

,NEIGHBOURHOOD,YEAR,Crime_Count
0,Arbutus Ridge,1999,616.000000
1,Arbutus Ridge,2000,648.000000
2,Arbutus Ridge,2001,619.000000
3,Arbutus Ridge,2002,441.000000
4,Arbutus Ridge,2003,361.000000
...,...,...,...
370,West Point Grey,2009,279.000000
371,West Point Grey,2010,365.000000
372,West Point Grey,2011,363.000000
373,West Point Grey,2012,356.263409


In [ ]:
from google.colab import files
# Save the combined historical and SARIMAX forecasted DataFrame to a CSV file.
Past_and_Future_df_yearly_neighbourhood_crime_count_sarimax.to_csv('Past_and_Future_df_yearly_neighbourhood_crime_count_sarimax.csv', index=False)
# Initiate the download of the CSV file to the local machine.
files.download('Past_and_Future_df_yearly_neighbourhood_crime_count_sarimax.csv')

# *Visualisations Based on the Forecast of the Sarimax Model*

In [ ]:
# Group the processed SARIMAX predictions (`final_predictions1`) by 'NEIGHBOURHOOD' and 'YEAR' to get future yearly crime counts.
Yearly_neighbourhood_crime_count_future = final_predictions1.groupby(['NEIGHBOURHOOD', 'YEAR'])['Crime Count'].sum().reset_index()
# Sort the future crime count DataFrame.
Yearly_neighbourhood_crime_count_future.sort_values(by=['NEIGHBOURHOOD', 'YEAR'], ascending=[True, True], inplace=True)
# Reset the index.
Yearly_neighbourhood_crime_count_future.reset_index(drop=True, inplace=True)
# Display the future crime counts.
Yearly_neighbourhood_crime_count_future

# Group the original `Train_df` by 'NEIGHBOURHOOD' and 'YEAR' to get past yearly crime counts.
Yearly_neighbourhood_crime_count_past = Train_df.groupby(['NEIGHBOURHOOD', 'YEAR']).size().reset_index(name='Crime Count')
# Sort the past crime count DataFrame.
Yearly_neighbourhood_crime_count_past.sort_values(by=['NEIGHBOURHOOD', 'YEAR'], ascending=[True, True], inplace=True)
# Reset the index.
Yearly_neighbourhood_crime_count_past.reset_index(drop=True, inplace=True)

# Concatenate past and future yearly crime counts.
df = pd.concat([Yearly_neighbourhood_crime_count_past, Yearly_neighbourhood_crime_count_future], ignore_index=True, axis=0)

# Create a line plot to visualize crime counts over past and future years per neighborhood (SARIMAX forecast).
fig = px.line(df, x='YEAR', y='Crime Count', title='Crime Count Over the Years(Past & Future) per Neighbourhood', color='NEIGHBOURHOOD', labels={'YEAR': 'Year'})
# Update plot layout for aesthetics.
fig.update_layout(xaxis_title='Year', yaxis_title='Crime Count', title_x = 0.5, title_font_size=24, title_font_family='Arial Black', width=1200, height=700)
# Display the plot.
fig.show()

***`Trend Observed:`***

Neighbourhood crime patterns continue into forecast period.

***`Key Insight:`***

SARIMAX effectively captures neighbourhood-level temporal trends.

In [ ]:
# Group the processed SARIMAX predictions (`final_predictions1`) by 'NEIGHBOURHOOD' and 'YEAR' to get future yearly crime counts.
Yearly_neighbourhood_crime_count_future = final_predictions1.groupby(['NEIGHBOURHOOD', 'YEAR'])['Crime Count'].sum().reset_index()
# Sort the future crime count DataFrame.
Yearly_neighbourhood_crime_count_future.sort_values(by=['NEIGHBOURHOOD', 'YEAR'], ascending=[True, True], inplace=True)
# Reset the index.
Yearly_neighbourhood_crime_count_future.reset_index(drop=True, inplace=True)
# Display the future crime counts.
Yearly_neighbourhood_crime_count_future

# Group the original `Train_df` by 'NEIGHBOURHOOD' and 'YEAR' to get past yearly crime counts.
Yearly_neighbourhood_crime_count_past = Train_df.groupby(['NEIGHBOURHOOD', 'YEAR']).size().reset_index(name='Crime Count')
# Sort the past crime count DataFrame.
Yearly_neighbourhood_crime_count_past.sort_values(by=['NEIGHBOURHOOD', 'YEAR'], ascending=[True, True], inplace=True)
# Reset the index.
Yearly_neighbourhood_crime_count_past.reset_index(drop=True, inplace=True)

# Concatenate past and future yearly crime counts.
df = pd.concat([Yearly_neighbourhood_crime_count_past, Yearly_neighbourhood_crime_count_future], ignore_index=True, axis=0)

# Create a density heatmap to visualize yearly crime counts per neighborhood for past and future years.
fig = px.density_heatmap(df, x='YEAR', y='NEIGHBOURHOOD', z='Crime Count', title='Yearly Crime Count per Neighbourhood Of Past & Future Years', labels={'YEAR': 'Year', 'NEIGHBOURHOOD': 'Neighbourhood'})
# Update plot layout for aesthetics.
fig.update_layout(title_x = 0.5, title=dict(font=dict(size=24, family='Arial Black')), width=1200, height=700)
# Set the x-axis type to 'category' to treat years as distinct categories.
fig.update_xaxes(type='category')
# Customize hover information to show Month (incorrect, should be Year), Year (incorrect, should be Neighborhood), and Crime Count.
# Note: The hover template contains a slight error from copy-paste; 'Month' and 'Year' labels should be swapped or re-evaluated for clarity.
fig.update_traces(hovertemplate='Month: %{x}<br>Year: %{y}<br>Crime Count: %{z}', selector=dict(type='heatmap'))
# Display the plot.
fig.show()

***`Trend Observed:`***

Crime density remains concentrated in specific neighbourhoods.

***`Key Insight:`***

Geographical crime patterns persist into forecast period.

# *`SARIMAX Model Conclusion:`*

In this project, the SARIMAX model was implemented to forecast yearly crime counts at the neighbourhood level. The model successfully captured long-term crime trends and temporal dependencies, producing stable and interpretable forecasts.

SARIMAX proved particularly effective for aggregated yearly data, where seasonal fluctuations are less volatile and long-term patterns dominate. Its structured time-series framework allowed for reliable modeling of neighbourhood-level crime progression and provided valuable insight into how crime trends evolve over time.

However, compared to machine learning approaches such as LightGBM and XGBoost, SARIMAX is limited in its ability to model complex nonlinear relationships and leverage extensive feature engineering. While it performs well for structured and interpretable forecasting, it is less flexible for high-frequency or multi-feature forecasting tasks.

Overall, SARIMAX served as a strong statistical baseline model in this project, effectively modeling yearly neighbourhood crime trends and complementing the more flexible machine learning approaches used for monthly crime forecasting.

# **Final Conclusion On The Overall Project:**

# *Overall Visual Pattern Summary:*

The visualizations confirm that crime patterns are:

Temporally structured

Seasonally predictable

Crime-type dependent

Geographically concentrated

Forecastable using machine learning and time-series models

# *Final Visual Insight Conclusion:*

The visualization analysis reveals that crime patterns exhibit strong temporal seasonality, crime-type dependency, and geographical concentration. Certain crime types and neighbourhoods consistently contribute higher crime volumes, while seasonal peaks occur predictably across years. Forecast visualizations from LightGBM, XGBoost, and SARIMAX models confirm the continuation of these historical patterns into future periods, validating the reliability of the forecasting models. These findings demonstrate that crime trends are structured and predictable, enabling data-driven decision-making for proactive crime prevention and resource allocation.

# *Final Conclusion On The Models:*

Overall, the implementation and evaluation of LightGBM, XGBoost, and SARIMAX models demonstrated the effectiveness of both machine learning and statistical approaches in forecasting crime trends at different temporal levels. LightGBM emerged as the best-performing model for monthly crime type forecasting due to its superior ability to capture seasonal patterns, temporal dependencies, and nonlinear relationships, while also providing stable and computationally efficient predictions. XGBoost also delivered strong and reliable performance, effectively modeling crime trends and seasonality, though with slightly lower efficiency and stability compared to LightGBM. SARIMAX performed well in forecasting yearly crime counts at the neighbourhood level, successfully capturing long-term trends and producing interpretable and consistent forecasts. While SARIMAX is well-suited for aggregated time-series forecasting, machine learning models such as LightGBM and XGBoost proved more powerful and flexible for high-frequency, feature-rich crime prediction tasks. Overall, the combined use of these models provided a comprehensive and reliable forecasting framework, demonstrating that advanced predictive modeling techniques can effectively support crime trend analysis and data-driven decision-making.

This project demonstrates how machine learning and time series forecasting can help improve crime prediction and support data-driven decision making.